##### https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter

In [103]:
import numpy as np
import pandas as pd

import re
import nltk

import spacy
import string

from nltk.corpus import stopwords
from collections import Counter

from nltk.stem import PorterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from bs4 import BeautifulSoup
from spellchecker import SpellChecker


pd.options.mode.chained_assignment = None

In [ ]:
full_df = pd.read_csv("twcs/twcs.csv", nrows=5000)

In [104]:
df = full_df[["text"]]
df["text"] = df["text"].astype(str)
full_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


## Lower Casing

Lower casing is a common text preprocessing technique. The idea is to convert the input text into same casing format so that 'text', 'Text' and 'TEXT' are treated the same way. 

This is more helpful for text featurization techniques like frequency, tfidf as it helps to combine the same words together thereby reducing the duplication and get correct counts / tfidf values.

This may not be helpful when we do tasks like Part of Speech tagging (where proper casing gives some information about Nouns and so on) and Sentiment Analysis (where upper casing refers to anger and so on)

By default, lower casing is done my most of the modern day vecotirzers and tokenizers like [sklearn TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and [Keras Tokenizer](https://keras.io/preprocessing/text/). So we need to set them to false as needed depending on our use case. 

In [105]:
df["text_lower"] = df["text"].str.lower()
df.head()

,text,text_lower
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.


In [ ]:
# drop the new column created in last cell
df.drop(["text_lower"], axis=1, inplace=True)

## Removal of Punctuations

One another common text preprocessing technique is to remove the punctuations from the text data. 

This is again a text standardization process that will help to treat 'hurray' and 'hurray!' in the same way.

We also need to carefully choose the list of punctuations to exclude depending on the use case. 

For example, the `string.punctuation` in python contains the following punctuation symbols 

`!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~`

We can add or remove more punctuations as per our need.

In [ ]:
PUNCT_TO_REMOVE = string.punctuation
print(PUNCT_TO_REMOVE)

In [106]:
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans("", "", PUNCT_TO_REMOVE))


df["text_wo_punct"] = df["text"].apply(lambda text: remove_punctuation(text))
df.head()

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


,text,text_wo_punct
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...
4,@sprintcare I did.,sprintcare I did


In [ ]:
for i in range(len(df["text_wo_punct"])):
    df.loc[i, "text_wo_punct"] = remove_punctuation(df.loc[i, "text"])

In [51]:
df["text"][0]

'@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.'

In [52]:
df["text_wo_punct"][0]

'115712 I understand I would like to assist you We would need to get you into a private secured link to further assist'

## Removal of stopwords

Stopwords are commonly occuring words in a language like 'the', 'a' and so on. 

They can be removed from the text most of the times, as they don't provide valuable information for downstream analysis. 

In cases like Part of Speech tagging, we should not remove them as provide very valuable information about the POS.

These stopword lists are already compiled for different languages and we can safely use them. 

For example, the stopword list for english language from the nltk package can be seen below.


In [53]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\original\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [108]:
stopwords.words("arabic")

['إذ',
 'إذا',
 'إذما',
 'إذن',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'إلا',
 'التي',
 'الذي',
 'الذين',
 'اللاتي',
 'اللائي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'أم',
 'أما',
 'أما',
 'إما',
 'أن',
 'إن',
 'إنا',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'إنما',
 'إنه',
 'أنى',
 'أنى',
 'آه',
 'آها',
 'أو',
 'أولاء',
 'أولئك',
 'أوه',
 'آي',
 'أي',
 'أيها',
 'إي',
 'أين',
 'أين',
 'أينما',
 'إيه',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بين',
 'بيد',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل'

Similarly we can also get the list for other languages as well and use them. 

In [55]:
STOPWORDS = set(stopwords.words("english"))


def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

,text,text_wo_punct,text_wo_stop
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...
4,@sprintcare I did.,sprintcare I did,sprintcare I


In [ ]:
df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

### Removal of Frequent words

- In the previos preprocessing step, we removed the stopwords based on language information. 

- But say, if we have a domain specific corpus, we might also have some frequent words which are of not so much importance to us. 

- So this step is to remove the frequent words in the given corpus. 

- If we use something like tfidf, this is automatically taken care of.  

- Let us get the most common words adn then remove them in the next step

**FREQWORDS = set([w for (w, wc) in cnt.most_common(10)]):**

This line creates a set called FREQWORDS that contains the most common 10 words in the corpus. 

It assumes that a variable cnt has been previously defined, likely as a Counter object, to count word frequencies in the text.

**cnt.most_common(10)** 

returns a list of tuples containing the most common words and their respective word counts in descending order.


The list comprehension [w for (w, wc) in cnt.most_common(10)] extracts only the words (not their counts) from the list of tuples.


Finally, set(...) converts this list of words into a set, which makes word lookup more efficient later in the code.
def remove_freqwords(text): This is the beginning of a function definition named remove_freqwords. It takes one argument, text, which is assumed to be a string.

"return " ".join([word for word in str(text).split() if word not in FREQWORDS])": This line is the implementation of the remove_freqwords function. It takes the input text, splits it into words, and then uses a list comprehension to include only the words that are not in the FREQWORDS set. Finally, it joins these words back into a string with spaces in between, effectively removing the frequent words.

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text)): This line creates a new column in the DataFrame df named "text_wo_stopfreq". It applies the remove_freqwords function to each element in the "text_wo_stop" column of the DataFrame using the apply() method. As a result, this new column will contain the text from "text_wo_stop" with the frequent words removed.

df.head(): Finally, this line displays the first few rows of the DataFrame df to inspect the result of removing frequent words.

The importance of this code lies in the following:

Text Cleaning: It's crucial to remove frequent words (commonly called "stopwords") in text preprocessing because they often don't carry much specific meaning and can introduce noise into NLP tasks.

Improved Analysis: By removing frequent words, the text becomes more focused on unique and potentially more informative content. This is especially valuable for text analysis tasks like sentiment analysis or topic modeling.

Reduced Dimensionality: In some cases, removing frequent words can help reduce the dimensionality of the data, making it easier and more efficient to work with in machine learning models.

Customization: This code allows you to customize the list of frequent words based on your specific dataset and needs. By adjusting the threshold (in this case, the top 10 words), you can control which words are considered "frequent."

In [56]:
cnt = Counter()
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('I', 1437),
 ('us', 752),
 ('DM', 514),
 ('help', 479),
 ('Please', 376),
 ('We', 338),
 ('Hi', 293),
 ('Thanks', 287),
 ('get', 279),
 ('please', 247)]

In [ ]:
map = {
    "i": 2,
    "ok": 20,
}

for i in df["text_wo_stop"].values:
    for j in i.split():
        if map[j] != None:
            map[j] = map[j] + 1
        else:
            map.update({j: 1})

In [57]:
FREQWORDS = set()
most_common = cnt.most_common(10)
for w, wc in most_common:
    FREQWORDS.add(w)


def remove_freqwords(text):
    """Custom function to remove the frequent words"""
    # Split the input text into words
    words = str(text).split()
    # Initialize an empty list to store words that are not in FREQWORDS
    filtered_words = []
    # Iterate through the words in the input text
    for word in words:
        # Check if the word is not in FREQWORDS
        if word not in FREQWORDS:
            # If it's not in FREQWORDS, add it to the filtered_words list
            filtered_words.append(word)
    # Join the filtered words into a single string with spaces in between

    cleaned_text = " ".join(filtered_words)
    return cleaned_text


df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

,text,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare I did,sprintcare I,sprintcare


In [ ]:
df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

## Removal of Rare words

This is very similar to previous preprocessing step but we will remove the rare words from the corpus. 

In [58]:
n_rare_words = 10

# Get the word frequency counts in descending order
most_common_words = cnt.most_common()

# Initialize an empty set to store rare words
RAREWORDS = set()

# Calculate the index for selecting the least common words
index_to_select = len(most_common_words) - n_rare_words

# Iterate through the most common words in descending order
for i, (word, wc) in enumerate(most_common_words):
    # Check if the current word should be considered rare
    if i >= index_to_select:
        RAREWORDS.add(word)


def remove_rarewords(text):
    """Custom function to remove the rare words"""
    # Split the input text into words
    words = str(text).split()
    # Initialize an empty list to store words that are not in RAREWORDS
    filtered_words = []
    # Iterate through the words in the input text
    for word in words:
        # Check if the word is not in RAREWORDS
        if word not in RAREWORDS:
            # If it's not in RAREWORDS, add it to the filtered_words list
            filtered_words.append(word)

    # Join the filtered words into a single string with spaces in between
    cleaned_text = " ".join(filtered_words)

    # Return the cleaned text
    return cleaned_text


df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(
    lambda text: remove_rarewords(text)
)
df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare
0,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 send Private Message assist Just click ...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare,sprintcare


In [ ]:
# Drop the two columns
df.drop(["text_wo_stopfreq", "text_wo_stopfreqrare"], axis=1, inplace=True)

## Stemming

- Stemming is the process of reducing inflected (or sometimes derived) words to their word stem, base or root form (From [Wikipedia](https://en.wikipedia.org/wiki/Stemming))

- For example, if there are two words in the corpus `walks` and `walking`, then stemming will stem the suffix to make them `walk`. But say in another example, we have two words `console` and `consoling`, the stemmer will remove the suffix and make them `consol` which is not a proper english word.

- There are several type of stemming algorithms available and one of the famous one is porter stemmer which is widely used. We can use nltk package for the same.

In [59]:
stemmer = PorterStemmer()


def stem_words(text):
    """Custom function to stem words in text"""
    # Split the input text into words
    words = text.split()
    # Initialize an empty list to store stemmed words
    stemmed_words = []
    # Iterate through the words in the input text
    for word in words:
        # Apply stemming to the word and add the stemmed word to the list
        stemmed_word = stemmer.stem(word)
        stemmed_words.append(stemmed_word)
    # Join the stemmed words into a single string with spaces in between
    stemmed_text = " ".join(stemmed_words)
    # Return the stemmed text
    return stemmed_text


df["text_stemmed"] = df["text"].apply(lambda text: stem_words(text))
df.head()

,text,text_stemmed
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcar and how do you propos we do that
2,@sprintcare I have sent several private messag...,@sprintcar i have sent sever privat messag and...
3,@115712 Please send us a Private Message so th...,@115712 pleas send us a privat messag so that ...
4,@sprintcare I did.,@sprintcar i did.


We can see that words like `private` and `propose` have their `e` at the end chopped off due to stemming. This is not intented. What can we do fort hat? We can use Lemmatization in such cases.

Also this porter stemmer is for English language. If we are working with other languages, we can use snowball stemmer. The supported languages for snowball stemmer are

In [60]:
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

## Lemmatization

Lemmatization is similar to stemming in reducing inflected words to their word stem but differs in the way that it makes sure the root word (also called as lemma) belongs to the language. 

As a result, this one is generally slower than stemming process. So depending on the speed requirement, we can choose to use either stemming or lemmatization. 

Let us use the `WordNetLemmatizer` in nltk to lemmatize our sentences

In [61]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\original\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
# Import the WordNet Lemmatizer
from nltk.stem import WordNetLemmatizer

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()


def lemmatize_words(text):
    """Custom function to lemmatize words in text"""
    # Split the input text into words
    words = text.split()
    # Initialize an empty list to store lemmatized words
    lemmatized_words = []
    # Iterate through the words in the input text
    for word in words:
        # Apply lemmatization to the word and add the lemmatized word to the list
        lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_words.append(lemmatized_word)
    # Join the lemmatized words into a single string with spaces in between
    lemmatized_text = " ".join(lemmatized_words)
    # Return the lemmatized text
    return lemmatized_text


df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

,text,text_stemmed,text_lemmatized
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcar and how do you propos we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcar i have sent sever privat messag and...,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 pleas send us a privat messag so that ...,@115712 Please send u a Private Message so tha...
4,@sprintcare I did.,@sprintcar i did.,@sprintcare I did.


We can see that the trailing `e` in the `propose` and `private` is retained when we use lemmatization unlike stemming. 

Wait. There is one more thing in lemmatization. Let us try to lemmatize `running` now.

**Example 1: Noun (Before and After Lemmatization)**

- Input Word: "cats" (Noun)
- Penn Treebank POS Tag: 'N'
- Lemmatized Word: "cat" (Noun, base form)


**Example 2: Verb (Before and After Lemmatization)**

- Input Word: "running" (Verb)
- Penn Treebank POS Tag: 'V'
- Lemmatized Word: "run" (Verb, base form)


**Example 3: Adverb (Before and After Lemmatization)**

- Input Word: "quickly" (Adverb)
- Penn Treebank POS Tag: 'R'
- Lemmatized Word: "quickly" (Adverb, no change)


**Example 4: Adjective (Before and After Lemmatization)**

- Input Word: "better" (Adjective)
- Penn Treebank POS Tag: 'J'
- Lemmatized Word: "good" (Adjective, base form)


**Example 5: Unknown or Non-Standard POS Tag (Before and After Lemmatization)**

- Input Word: "unknown" (Unknown)
- Penn Treebank POS Tag: 'X' (Non-standard)
- Lemmatized Word: "unknown" (Noun, fallback to default)

In [63]:
lemmatizer.lemmatize("running")

'running'

Wow. It returned `running` as such without converting it to the root form `run`. This is because the lemmatization process depends on the POS tag to come up with the correct lemma. Now let us lemmatize again by providing the POS tag for the word.

In [68]:
lemmatizer.lemmatize("running", "v")  # v for verb

'run'

Now we are getting the root form `run`. So we also need to provide the POS tag of the word along with the word for lemmatizer in nltk. Depending on the POS, the lemmatizer may return different results.

Let us take the example, `stripes` and check the lemma when it is both verb and noun.

In [69]:
print("Word is : stripes")
print("Lemma result for verb : ", lemmatizer.lemmatize("stripes", "v"))
print("Lemma result for noun : ", lemmatizer.lemmatize("stripes", "n"))

Word is : stripes
Lemma result for verb :  strip
Lemma result for noun :  stripe


Now let us redo the lemmatization process for our dataset.

In [70]:
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\original\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()


# Define a mapping from Penn Treebank POS tags to WordNet POS tags
wordnet_map = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}


def lemmatize_words(text):
    """Custom function to lemmatize words in text with POS tagging"""
    # Tokenize the input text and perform POS tagging
    pos_tagged_text = nltk.pos_tag(text.split())
    # Initialize an empty list to store lemmatized words
    lemmatized_words = []
    # Iterate through the words and their POS tags in the input text
    for word, pos in pos_tagged_text:
        # Map the POS tag to a WordNet POS tag using the wordnet_map dictionary
        wordnet_pos = wordnet_map.get(pos[0], wordnet.NOUN)
        # Apply lemmatization to the word with the specified POS tag
        lemmatized_word = lemmatizer.lemmatize(word, wordnet_pos)
        # Add the lemmatized word to the list
        lemmatized_words.append(lemmatized_word)

    # Join the lemmatized words into a single string with spaces in between
    lemmatized_text = " ".join(lemmatized_words)
    # Return the lemmatized text
    return lemmatized_text

In [71]:
df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

,text,text_stemmed,text_lemmatized
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcar and how do you propos we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcar i have sent sever privat messag and...,@sprintcare I have send several private messag...
3,@115712 Please send us a Private Message so th...,@115712 pleas send us a privat messag so that ...,@115712 Please send u a Private Message so tha...
4,@sprintcare I did.,@sprintcar i did.,@sprintcare I did.


We can now see that in the third row, `sent` got converted to `send` since we provided the POS tag for lemmatization.

## Removal of Emojis

With more and more usage of social media platforms, there is an explosion in the usage of emojis in our day to day life as well. Probably we might need to remove these emojis for some of our textual analysis.

Thanks to [this code,](https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b) please find below a helper function to remove emojis from our text. 

In [72]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)


remove_emoji("game is on 🔥🔥")

'game is on '

In [73]:
remove_emoji("Hilarious😂")

'Hilarious'

## Removal of Emoticons

- This is what we did in the last step right? Nope. We did remove emojis in the last step but not emoticons. 

- There is a minor difference between emojis and emoticons. 

- From Grammarist.com, emoticon is built from keyboard characters that when put together in a certain way represent a facial expression, an emoji is an actual image.

:-) is an emoticon

😀 is an emoji

- Thanks to [NeelShah](https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py) for the wonderful collection of emoticons, we are going to use them to remove emoticons. 

- Please note again that the removal of emojis / emoticons are not always preferred and decision should be made based on the use case at hand.

In [74]:
# Thanks : https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py
EMOTICONS = {
    ":‑\)": "Happy face or smiley",
    ":\)": "Happy face or smiley",
    ":-\]": "Happy face or smiley",
    ":\]": "Happy face or smiley",
    ":-3": "Happy face smiley",
    ":3": "Happy face smiley",
    ":->": "Happy face smiley",
    ":>": "Happy face smiley",
    "8-\)": "Happy face smiley",
    ":o\)": "Happy face smiley",
    ":-\}": "Happy face smiley",
    ":\}": "Happy face smiley",
    ":-\)": "Happy face smiley",
    ":c\)": "Happy face smiley",
    ":\^\)": "Happy face smiley",
    "=\]": "Happy face smiley",
    "=\)": "Happy face smiley",
    ":‑D": "Laughing, big grin or laugh with glasses",
    ":D": "Laughing, big grin or laugh with glasses",
    "8‑D": "Laughing, big grin or laugh with glasses",
    "8D": "Laughing, big grin or laugh with glasses",
    "X‑D": "Laughing, big grin or laugh with glasses",
    "XD": "Laughing, big grin or laugh with glasses",
    "=D": "Laughing, big grin or laugh with glasses",
    "=3": "Laughing, big grin or laugh with glasses",
    "B\^D": "Laughing, big grin or laugh with glasses",
    ":-\)\)": "Very happy",
    ":‑\(": "Frown, sad, andry or pouting",
    ":-\(": "Frown, sad, andry or pouting",
    ":\(": "Frown, sad, andry or pouting",
    ":‑c": "Frown, sad, andry or pouting",
    ":c": "Frown, sad, andry or pouting",
    ":‑<": "Frown, sad, andry or pouting",
    ":<": "Frown, sad, andry or pouting",
    ":‑\[": "Frown, sad, andry or pouting",
    ":\[": "Frown, sad, andry or pouting",
    ":-\|\|": "Frown, sad, andry or pouting",
    ">:\[": "Frown, sad, andry or pouting",
    ":\{": "Frown, sad, andry or pouting",
    ":@": "Frown, sad, andry or pouting",
    ">:\(": "Frown, sad, andry or pouting",
    ":'‑\(": "Crying",
    ":'\(": "Crying",
    ":'‑\)": "Tears of happiness",
    ":'\)": "Tears of happiness",
    "D‑':": "Horror",
    "D:<": "Disgust",
    "D:": "Sadness",
    "D8": "Great dismay",
    "D;": "Great dismay",
    "D=": "Great dismay",
    "DX": "Great dismay",
    ":‑O": "Surprise",
    ":O": "Surprise",
    ":‑o": "Surprise",
    ":o": "Surprise",
    ":-0": "Shock",
    "8‑0": "Yawn",
    ">:O": "Yawn",
    ":-\*": "Kiss",
    ":\*": "Kiss",
    ":X": "Kiss",
    ";‑\)": "Wink or smirk",
    ";\)": "Wink or smirk",
    "\*-\)": "Wink or smirk",
    "\*\)": "Wink or smirk",
    ";‑\]": "Wink or smirk",
    ";\]": "Wink or smirk",
    ";\^\)": "Wink or smirk",
    ":‑,": "Wink or smirk",
    ";D": "Wink or smirk",
    ":‑P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "X‑P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "XP": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":‑Þ": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":Þ": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":b": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "d:": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "=p": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ">:P": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    ":‑/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":-[.]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ">:[(\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ">:/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":[(\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    "=/": "Skeptical, annoyed, undecided, uneasy or hesitant",
    "=[(\\\)]": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":L": "Skeptical, annoyed, undecided, uneasy or hesitant",
    "=L": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":S": "Skeptical, annoyed, undecided, uneasy or hesitant",
    ":‑\|": "Straight face",
    ":\|": "Straight face",
    ":$": "Embarrassed or blushing",
    ":‑x": "Sealed lips or wearing braces or tongue-tied",
    ":x": "Sealed lips or wearing braces or tongue-tied",
    ":‑#": "Sealed lips or wearing braces or tongue-tied",
    ":#": "Sealed lips or wearing braces or tongue-tied",
    ":‑&": "Sealed lips or wearing braces or tongue-tied",
    ":&": "Sealed lips or wearing braces or tongue-tied",
    "O:‑\)": "Angel, saint or innocent",
    "O:\)": "Angel, saint or innocent",
    "0:‑3": "Angel, saint or innocent",
    "0:3": "Angel, saint or innocent",
    "0:‑\)": "Angel, saint or innocent",
    "0:\)": "Angel, saint or innocent",
    ":‑b": "Tongue sticking out, cheeky, playful or blowing a raspberry",
    "0;\^\)": "Angel, saint or innocent",
    ">:‑\)": "Evil or devilish",
    ">:\)": "Evil or devilish",
    "\}:‑\)": "Evil or devilish",
    "\}:\)": "Evil or devilish",
    "3:‑\)": "Evil or devilish",
    "3:\)": "Evil or devilish",
    ">;\)": "Evil or devilish",
    "\|;‑\)": "Cool",
    "\|‑O": "Bored",
    ":‑J": "Tongue-in-cheek",
    "#‑\)": "Party all night",
    "%‑\)": "Drunk or confused",
    "%\)": "Drunk or confused",
    ":-###..": "Being sick",
    ":###..": "Being sick",
    "<:‑\|": "Dump",
    "\(>_<\)": "Troubled",
    "\(>_<\)>": "Troubled",
    "\(';'\)": "Baby",
    "\(\^\^>``": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\(\^_\^;\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\(-_-;\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\(~_~;\) \(・\.・;\)": "Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    "\(-_-\)zzz": "Sleeping",
    "\(\^_-\)": "Wink",
    "\(\(\+_\+\)\)": "Confused",
    "\(\+o\+\)": "Confused",
    "\(o\|o\)": "Ultraman",
    "\^_\^": "Joyful",
    "\(\^_\^\)/": "Joyful",
    "\(\^O\^\)／": "Joyful",
    "\(\^o\^\)／": "Joyful",
    "\(__\)": "Kowtow as a sign of respect, or dogeza for apology",
    "_\(\._\.\)_": "Kowtow as a sign of respect, or dogeza for apology",
    "<\(_ _\)>": "Kowtow as a sign of respect, or dogeza for apology",
    "<m\(__\)m>": "Kowtow as a sign of respect, or dogeza for apology",
    "m\(__\)m": "Kowtow as a sign of respect, or dogeza for apology",
    "m\(_ _\)m": "Kowtow as a sign of respect, or dogeza for apology",
    "\('_'\)": "Sad or Crying",
    "\(/_;\)": "Sad or Crying",
    "\(T_T\) \(;_;\)": "Sad or Crying",
    "\(;_;": "Sad of Crying",
    "\(;_:\)": "Sad or Crying",
    "\(;O;\)": "Sad or Crying",
    "\(:_;\)": "Sad or Crying",
    "\(ToT\)": "Sad or Crying",
    ";_;": "Sad or Crying",
    ";-;": "Sad or Crying",
    ";n;": "Sad or Crying",
    ";;": "Sad or Crying",
    "Q\.Q": "Sad or Crying",
    "T\.T": "Sad or Crying",
    "QQ": "Sad or Crying",
    "Q_Q": "Sad or Crying",
    "\(-\.-\)": "Shame",
    "\(-_-\)": "Shame",
    "\(一一\)": "Shame",
    "\(；一_一\)": "Shame",
    "\(=_=\)": "Tired",
    "\(=\^\·\^=\)": "cat",
    "\(=\^\·\·\^=\)": "cat",
    "=_\^=	": "cat",
    "\(\.\.\)": "Looking down",
    "\(\._\.\)": "Looking down",
    "\^m\^": "Giggling with hand covering mouth",
    "\(\・\・?": "Confusion",
    "\(?_?\)": "Confusion",
    ">\^_\^<": "Normal Laugh",
    "<\^!\^>": "Normal Laugh",
    "\^/\^": "Normal Laugh",
    "\（\*\^_\^\*）": "Normal Laugh",
    "\(\^<\^\) \(\^\.\^\)": "Normal Laugh",
    "\(^\^\)": "Normal Laugh",
    "\(\^\.\^\)": "Normal Laugh",
    "\(\^_\^\.\)": "Normal Laugh",
    "\(\^_\^\)": "Normal Laugh",
    "\(\^\^\)": "Normal Laugh",
    "\(\^J\^\)": "Normal Laugh",
    "\(\*\^\.\^\*\)": "Normal Laugh",
    "\(\^—\^\）": "Normal Laugh",
    "\(#\^\.\^#\)": "Normal Laugh",
    "\（\^—\^\）": "Waving",
    "\(;_;\)/~~~": "Waving",
    "\(\^\.\^\)/~~~": "Waving",
    "\(-_-\)/~~~ \($\·\·\)/~~~": "Waving",
    "\(T_T\)/~~~": "Waving",
    "\(ToT\)/~~~": "Waving",
    "\(\*\^0\^\*\)": "Excited",
    "\(\*_\*\)": "Amazed",
    "\(\*_\*;": "Amazed",
    "\(\+_\+\) \(@_@\)": "Amazed",
    "\(\*\^\^\)v": "Laughing,Cheerful",
    "\(\^_\^\)v": "Laughing,Cheerful",
    "\(\(d[-_-]b\)\)": "Headphones,Listening to music",
    '\(-"-\)': "Worried",
    "\(ーー;\)": "Worried",
    "\(\^0_0\^\)": "Eyeglasses",
    "\(\＾ｖ\＾\)": "Happy",
    "\(\＾ｕ\＾\)": "Happy",
    "\(\^\)o\(\^\)": "Happy",
    "\(\^O\^\)": "Happy",
    "\(\^o\^\)": "Happy",
    "\)\^o\^\(": "Happy",
    ":O o_O": "Surprised",
    "o_0": "Surprised",
    "o\.O": "Surpised",
    "\(o\.o\)": "Surprised",
    "oO": "Surprised",
    "\(\*￣m￣\)": "Dissatisfied",
    "\(‘A`\)": "Snubbed or Deflated",
}

In [79]:
def remove_emoticons(text):
    """Custom function to remove emoticons from text"""
    # Create a regular expression pattern for emoticons
    emoticon_pattern = re.compile("(" + "|".join(k for k in EMOTICONS) + ")")
    # Use the re.sub() method to replace emoticons with an empty string
    cleaned_text = emoticon_pattern.sub(r"", text)
    # Return the cleaned text
    return cleaned_text


remove_emoticons("Hello :-)")

'Hello '

In [80]:
remove_emoticons("I am sad :(")

'I am sad '

### Conversion of Emoticon to Words

- In the previous step, we have removed the emoticons. In case of use cases like sentiment analysis, the emoticons give some valuable information and so removing them might not be a good solution. What can we do in such cases?

- One way is to convert the emoticons to word format so that they can be used in downstream modeling processes. Thanks for Neel again for the wonderful dictionary that we have used in the previous step. We are going to use that again for conversion of emoticons to words. 


In [82]:
def convert_emoticons(text):
    """Custom function to convert emoticons to textual representations"""
    # Iterate through each emoticon in the EMOTICONS dictionary
    for emot in EMOTICONS:
        # Replace the emoticon in the text with its textual representation
        replacement = "_".join(EMOTICONS[emot].replace(",", "").split())
        text = re.sub("(" + emot + ")", replacement, text)
    # Return the text with converted emoticons
    return text


text = "Hello :-) :-)"
convert_emoticons(text)

'Hello Happy_face_smiley Happy_face_smiley'

In [83]:
text = "I am sad :()"
convert_emoticons(text)

'I am sad Frown_sad_andry_or_poutingConfusion'

In [84]:
text = "I am sad :("
convert_emoticons(text)

'I am sad Frown_sad_andry_or_pouting'

This method might be better for some use cases when we do not want to miss out on the emoticon information.

## Conversion of Emoji to Words

Now let us do the same for Emojis as well. Neel Shah has put together a list of emojis with the corresponding words as well as part of his [Github repo](https://github.com/NeelShah18/emot/blob/master/emot/emo_unicode.py). We are going to make use of this dictionary to convert the emojis to corresponding words.

Again this conversion might be better than emoji removal for certain use cases. Please use the one that is suitable for the use case. 

In [85]:
EMO_UNICODE = {
    ":1st_place_medal:": "\U0001F947",
    ":2nd_place_medal:": "\U0001F948",
    ":3rd_place_medal:": "\U0001F949",
    ":AB_button_(blood_type):": "\U0001F18E",
    ":ATM_sign:": "\U0001F3E7",
    ":A_button_(blood_type):": "\U0001F170",
    ":Afghanistan:": "\U0001F1E6 \U0001F1EB",
    ":Albania:": "\U0001F1E6 \U0001F1F1",
    ":Algeria:": "\U0001F1E9 \U0001F1FF",
    ":American_Samoa:": "\U0001F1E6 \U0001F1F8",
    ":Andorra:": "\U0001F1E6 \U0001F1E9",
    ":Angola:": "\U0001F1E6 \U0001F1F4",
    ":Anguilla:": "\U0001F1E6 \U0001F1EE",
    ":Antarctica:": "\U0001F1E6 \U0001F1F6",
    ":Antigua_&_Barbuda:": "\U0001F1E6 \U0001F1EC",
    ":Aquarius:": "\U00002652",
    ":Argentina:": "\U0001F1E6 \U0001F1F7",
    ":Aries:": "\U00002648",
    ":Armenia:": "\U0001F1E6 \U0001F1F2",
    ":Aruba:": "\U0001F1E6 \U0001F1FC",
    ":Ascension_Island:": "\U0001F1E6 \U0001F1E8",
    ":Australia:": "\U0001F1E6 \U0001F1FA",
    ":Austria:": "\U0001F1E6 \U0001F1F9",
    ":Azerbaijan:": "\U0001F1E6 \U0001F1FF",
    ":BACK_arrow:": "\U0001F519",
    ":B_button_(blood_type):": "\U0001F171",
    ":Bahamas:": "\U0001F1E7 \U0001F1F8",
    ":Bahrain:": "\U0001F1E7 \U0001F1ED",
    ":Bangladesh:": "\U0001F1E7 \U0001F1E9",
    ":Barbados:": "\U0001F1E7 \U0001F1E7",
    ":Belarus:": "\U0001F1E7 \U0001F1FE",
    ":Belgium:": "\U0001F1E7 \U0001F1EA",
    ":Belize:": "\U0001F1E7 \U0001F1FF",
    ":Benin:": "\U0001F1E7 \U0001F1EF",
    ":Bermuda:": "\U0001F1E7 \U0001F1F2",
    ":Bhutan:": "\U0001F1E7 \U0001F1F9",
    ":Bolivia:": "\U0001F1E7 \U0001F1F4",
    ":Bosnia_&_Herzegovina:": "\U0001F1E7 \U0001F1E6",
    ":Botswana:": "\U0001F1E7 \U0001F1FC",
    ":Bouvet_Island:": "\U0001F1E7 \U0001F1FB",
    ":Brazil:": "\U0001F1E7 \U0001F1F7",
    ":British_Indian_Ocean_Territory:": "\U0001F1EE \U0001F1F4",
    ":British_Virgin_Islands:": "\U0001F1FB \U0001F1EC",
    ":Brunei:": "\U0001F1E7 \U0001F1F3",
    ":Bulgaria:": "\U0001F1E7 \U0001F1EC",
    ":Burkina_Faso:": "\U0001F1E7 \U0001F1EB",
    ":Burundi:": "\U0001F1E7 \U0001F1EE",
    ":CL_button:": "\U0001F191",
    ":COOL_button:": "\U0001F192",
    ":Cambodia:": "\U0001F1F0 \U0001F1ED",
    ":Cameroon:": "\U0001F1E8 \U0001F1F2",
    ":Canada:": "\U0001F1E8 \U0001F1E6",
    ":Canary_Islands:": "\U0001F1EE \U0001F1E8",
    ":Cancer:": "\U0000264B",
    ":Cape_Verde:": "\U0001F1E8 \U0001F1FB",
    ":Capricorn:": "\U00002651",
    ":Caribbean_Netherlands:": "\U0001F1E7 \U0001F1F6",
    ":Cayman_Islands:": "\U0001F1F0 \U0001F1FE",
    ":Central_African_Republic:": "\U0001F1E8 \U0001F1EB",
    ":Ceuta_&_Melilla:": "\U0001F1EA \U0001F1E6",
    ":Chad:": "\U0001F1F9 \U0001F1E9",
    ":Chile:": "\U0001F1E8 \U0001F1F1",
    ":China:": "\U0001F1E8 \U0001F1F3",
    ":Christmas_Island:": "\U0001F1E8 \U0001F1FD",
    ":Christmas_tree:": "\U0001F384",
    ":Clipperton_Island:": "\U0001F1E8 \U0001F1F5",
    ":Cocos_(Keeling)_Islands:": "\U0001F1E8 \U0001F1E8",
    ":Colombia:": "\U0001F1E8 \U0001F1F4",
    ":Comoros:": "\U0001F1F0 \U0001F1F2",
    ":Congo_-_Brazzaville:": "\U0001F1E8 \U0001F1EC",
    ":Congo_-_Kinshasa:": "\U0001F1E8 \U0001F1E9",
    ":Cook_Islands:": "\U0001F1E8 \U0001F1F0",
    ":Costa_Rica:": "\U0001F1E8 \U0001F1F7",
    ":Croatia:": "\U0001F1ED \U0001F1F7",
    ":Cuba:": "\U0001F1E8 \U0001F1FA",
    ":Curaçao:": "\U0001F1E8 \U0001F1FC",
    ":Cyprus:": "\U0001F1E8 \U0001F1FE",
    ":Czech_Republic:": "\U0001F1E8 \U0001F1FF",
    ":Côte_d’Ivoire:": "\U0001F1E8 \U0001F1EE",
    ":Denmark:": "\U0001F1E9 \U0001F1F0",
    ":Diego_Garcia:": "\U0001F1E9 \U0001F1EC",
    ":Djibouti:": "\U0001F1E9 \U0001F1EF",
    ":Dominica:": "\U0001F1E9 \U0001F1F2",
    ":Dominican_Republic:": "\U0001F1E9 \U0001F1F4",
    ":END_arrow:": "\U0001F51A",
    ":Ecuador:": "\U0001F1EA \U0001F1E8",
    ":Egypt:": "\U0001F1EA \U0001F1EC",
    ":El_Salvador:": "\U0001F1F8 \U0001F1FB",
    ":Equatorial_Guinea:": "\U0001F1EC \U0001F1F6",
    ":Eritrea:": "\U0001F1EA \U0001F1F7",
    ":Estonia:": "\U0001F1EA \U0001F1EA",
    ":Ethiopia:": "\U0001F1EA \U0001F1F9",
    ":European_Union:": "\U0001F1EA \U0001F1FA",
    ":FREE_button:": "\U0001F193",
    ":Falkland_Islands:": "\U0001F1EB \U0001F1F0",
    ":Faroe_Islands:": "\U0001F1EB \U0001F1F4",
    ":Fiji:": "\U0001F1EB \U0001F1EF",
    ":Finland:": "\U0001F1EB \U0001F1EE",
    ":France:": "\U0001F1EB \U0001F1F7",
    ":French_Guiana:": "\U0001F1EC \U0001F1EB",
    ":French_Polynesia:": "\U0001F1F5 \U0001F1EB",
    ":French_Southern_Territories:": "\U0001F1F9 \U0001F1EB",
    ":Gabon:": "\U0001F1EC \U0001F1E6",
    ":Gambia:": "\U0001F1EC \U0001F1F2",
    ":Gemini:": "\U0000264A",
    ":Georgia:": "\U0001F1EC \U0001F1EA",
    ":Germany:": "\U0001F1E9 \U0001F1EA",
    ":Ghana:": "\U0001F1EC \U0001F1ED",
    ":Gibraltar:": "\U0001F1EC \U0001F1EE",
    ":Greece:": "\U0001F1EC \U0001F1F7",
    ":Greenland:": "\U0001F1EC \U0001F1F1",
    ":Grenada:": "\U0001F1EC \U0001F1E9",
    ":Guadeloupe:": "\U0001F1EC \U0001F1F5",
    ":Guam:": "\U0001F1EC \U0001F1FA",
    ":Guatemala:": "\U0001F1EC \U0001F1F9",
    ":Guernsey:": "\U0001F1EC \U0001F1EC",
    ":Guinea:": "\U0001F1EC \U0001F1F3",
    ":Guinea-Bissau:": "\U0001F1EC \U0001F1FC",
    ":Guyana:": "\U0001F1EC \U0001F1FE",
    ":Haiti:": "\U0001F1ED \U0001F1F9",
    ":Heard_&_McDonald_Islands:": "\U0001F1ED \U0001F1F2",
    ":Honduras:": "\U0001F1ED \U0001F1F3",
    ":Hong_Kong_SAR_China:": "\U0001F1ED \U0001F1F0",
    ":Hungary:": "\U0001F1ED \U0001F1FA",
    ":ID_button:": "\U0001F194",
    ":Iceland:": "\U0001F1EE \U0001F1F8",
    ":India:": "\U0001F1EE \U0001F1F3",
    ":Indonesia:": "\U0001F1EE \U0001F1E9",
    ":Iran:": "\U0001F1EE \U0001F1F7",
    ":Iraq:": "\U0001F1EE \U0001F1F6",
    ":Ireland:": "\U0001F1EE \U0001F1EA",
    ":Isle_of_Man:": "\U0001F1EE \U0001F1F2",
    ":Israel:": "\U0001F1EE \U0001F1F1",
    ":Italy:": "\U0001F1EE \U0001F1F9",
    ":Jamaica:": "\U0001F1EF \U0001F1F2",
    ":Japan:": "\U0001F1EF \U0001F1F5",
    ":Japanese_acceptable_button:": "\U0001F251",
    ":Japanese_application_button:": "\U0001F238",
    ":Japanese_bargain_button:": "\U0001F250",
    ":Japanese_castle:": "\U0001F3EF",
    ":Japanese_congratulations_button:": "\U00003297",
    ":Japanese_discount_button:": "\U0001F239",
    ":Japanese_dolls:": "\U0001F38E",
    ":Japanese_free_of_charge_button:": "\U0001F21A",
    ":Japanese_here_button:": "\U0001F201",
    ":Japanese_monthly_amount_button:": "\U0001F237",
    ":Japanese_no_vacancy_button:": "\U0001F235",
    ":Japanese_not_free_of_charge_button:": "\U0001F236",
    ":Japanese_open_for_business_button:": "\U0001F23A",
    ":Japanese_passing_grade_button:": "\U0001F234",
    ":Japanese_post_office:": "\U0001F3E3",
    ":Japanese_prohibited_button:": "\U0001F232",
    ":Japanese_reserved_button:": "\U0001F22F",
    ":Japanese_secret_button:": "\U00003299",
    ":Japanese_service_charge_button:": "\U0001F202",
    ":Japanese_symbol_for_beginner:": "\U0001F530",
    ":Japanese_vacancy_button:": "\U0001F233",
    ":Jersey:": "\U0001F1EF \U0001F1EA",
    ":Jordan:": "\U0001F1EF \U0001F1F4",
    ":Kazakhstan:": "\U0001F1F0 \U0001F1FF",
    ":Kenya:": "\U0001F1F0 \U0001F1EA",
    ":Kiribati:": "\U0001F1F0 \U0001F1EE",
    ":Kosovo:": "\U0001F1FD \U0001F1F0",
    ":Kuwait:": "\U0001F1F0 \U0001F1FC",
    ":Kyrgyzstan:": "\U0001F1F0 \U0001F1EC",
    ":Laos:": "\U0001F1F1 \U0001F1E6",
    ":Latvia:": "\U0001F1F1 \U0001F1FB",
    ":Lebanon:": "\U0001F1F1 \U0001F1E7",
    ":Leo:": "\U0000264C",
    ":Lesotho:": "\U0001F1F1 \U0001F1F8",
    ":Liberia:": "\U0001F1F1 \U0001F1F7",
    ":Libra:": "\U0000264E",
    ":Libya:": "\U0001F1F1 \U0001F1FE",
    ":Liechtenstein:": "\U0001F1F1 \U0001F1EE",
    ":Lithuania:": "\U0001F1F1 \U0001F1F9",
    ":Luxembourg:": "\U0001F1F1 \U0001F1FA",
    ":Macau_SAR_China:": "\U0001F1F2 \U0001F1F4",
    ":Macedonia:": "\U0001F1F2 \U0001F1F0",
    ":Madagascar:": "\U0001F1F2 \U0001F1EC",
    ":Malawi:": "\U0001F1F2 \U0001F1FC",
    ":Malaysia:": "\U0001F1F2 \U0001F1FE",
    ":Maldives:": "\U0001F1F2 \U0001F1FB",
    ":Mali:": "\U0001F1F2 \U0001F1F1",
    ":Malta:": "\U0001F1F2 \U0001F1F9",
    ":Marshall_Islands:": "\U0001F1F2 \U0001F1ED",
    ":Martinique:": "\U0001F1F2 \U0001F1F6",
    ":Mauritania:": "\U0001F1F2 \U0001F1F7",
    ":Mauritius:": "\U0001F1F2 \U0001F1FA",
    ":Mayotte:": "\U0001F1FE \U0001F1F9",
    ":Mexico:": "\U0001F1F2 \U0001F1FD",
    ":Micronesia:": "\U0001F1EB \U0001F1F2",
    ":Moldova:": "\U0001F1F2 \U0001F1E9",
    ":Monaco:": "\U0001F1F2 \U0001F1E8",
    ":Mongolia:": "\U0001F1F2 \U0001F1F3",
    ":Montenegro:": "\U0001F1F2 \U0001F1EA",
    ":Montserrat:": "\U0001F1F2 \U0001F1F8",
    ":Morocco:": "\U0001F1F2 \U0001F1E6",
    ":Mozambique:": "\U0001F1F2 \U0001F1FF",
    ":Mrs._Claus:": "\U0001F936",
    ":Mrs._Claus_dark_skin_tone:": "\U0001F936 \U0001F3FF",
    ":Mrs._Claus_light_skin_tone:": "\U0001F936 \U0001F3FB",
    ":Mrs._Claus_medium-dark_skin_tone:": "\U0001F936 \U0001F3FE",
    ":Mrs._Claus_medium-light_skin_tone:": "\U0001F936 \U0001F3FC",
    ":Mrs._Claus_medium_skin_tone:": "\U0001F936 \U0001F3FD",
    ":Myanmar_(Burma):": "\U0001F1F2 \U0001F1F2",
    ":NEW_button:": "\U0001F195",
    ":NG_button:": "\U0001F196",
    ":Namibia:": "\U0001F1F3 \U0001F1E6",
    ":Nauru:": "\U0001F1F3 \U0001F1F7",
    ":Nepal:": "\U0001F1F3 \U0001F1F5",
    ":Netherlands:": "\U0001F1F3 \U0001F1F1",
    ":New_Caledonia:": "\U0001F1F3 \U0001F1E8",
    ":New_Zealand:": "\U0001F1F3 \U0001F1FF",
    ":Nicaragua:": "\U0001F1F3 \U0001F1EE",
    ":Niger:": "\U0001F1F3 \U0001F1EA",
    ":Nigeria:": "\U0001F1F3 \U0001F1EC",
    ":Niue:": "\U0001F1F3 \U0001F1FA",
    ":Norfolk_Island:": "\U0001F1F3 \U0001F1EB",
    ":North_Korea:": "\U0001F1F0 \U0001F1F5",
    ":Northern_Mariana_Islands:": "\U0001F1F2 \U0001F1F5",
    ":Norway:": "\U0001F1F3 \U0001F1F4",
    ":OK_button:": "\U0001F197",
    ":OK_hand:": "\U0001F44C",
    ":OK_hand_dark_skin_tone:": "\U0001F44C \U0001F3FF",
    ":OK_hand_light_skin_tone:": "\U0001F44C \U0001F3FB",
    ":OK_hand_medium-dark_skin_tone:": "\U0001F44C \U0001F3FE",
    ":OK_hand_medium-light_skin_tone:": "\U0001F44C \U0001F3FC",
    ":OK_hand_medium_skin_tone:": "\U0001F44C \U0001F3FD",
    ":ON!_arrow:": "\U0001F51B",
    ":O_button_(blood_type):": "\U0001F17E",
    ":Oman:": "\U0001F1F4 \U0001F1F2",
    ":Ophiuchus:": "\U000026CE",
    ":P_button:": "\U0001F17F",
    ":Pakistan:": "\U0001F1F5 \U0001F1F0",
    ":Palau:": "\U0001F1F5 \U0001F1FC",
    ":Palestinian_Territories:": "\U0001F1F5 \U0001F1F8",
    ":Panama:": "\U0001F1F5 \U0001F1E6",
    ":Papua_New_Guinea:": "\U0001F1F5 \U0001F1EC",
    ":Paraguay:": "\U0001F1F5 \U0001F1FE",
    ":Peru:": "\U0001F1F5 \U0001F1EA",
    ":Philippines:": "\U0001F1F5 \U0001F1ED",
    ":Pisces:": "\U00002653",
    ":Pitcairn_Islands:": "\U0001F1F5 \U0001F1F3",
    ":Poland:": "\U0001F1F5 \U0001F1F1",
    ":Portugal:": "\U0001F1F5 \U0001F1F9",
    ":Puerto_Rico:": "\U0001F1F5 \U0001F1F7",
    ":Qatar:": "\U0001F1F6 \U0001F1E6",
    ":Romania:": "\U0001F1F7 \U0001F1F4",
    ":Russia:": "\U0001F1F7 \U0001F1FA",
    ":Rwanda:": "\U0001F1F7 \U0001F1FC",
    ":Réunion:": "\U0001F1F7 \U0001F1EA",
    ":SOON_arrow:": "\U0001F51C",
    ":SOS_button:": "\U0001F198",
    ":Sagittarius:": "\U00002650",
    ":Samoa:": "\U0001F1FC \U0001F1F8",
    ":San_Marino:": "\U0001F1F8 \U0001F1F2",
    ":Santa_Claus:": "\U0001F385",
    ":Santa_Claus_dark_skin_tone:": "\U0001F385 \U0001F3FF",
    ":Santa_Claus_light_skin_tone:": "\U0001F385 \U0001F3FB",
    ":Santa_Claus_medium-dark_skin_tone:": "\U0001F385 \U0001F3FE",
    ":Santa_Claus_medium-light_skin_tone:": "\U0001F385 \U0001F3FC",
    ":Santa_Claus_medium_skin_tone:": "\U0001F385 \U0001F3FD",
    ":Saudi_Arabia:": "\U0001F1F8 \U0001F1E6",
    ":Scorpius:": "\U0000264F",
    ":Senegal:": "\U0001F1F8 \U0001F1F3",
    ":Serbia:": "\U0001F1F7 \U0001F1F8",
    ":Seychelles:": "\U0001F1F8 \U0001F1E8",
    ":Sierra_Leone:": "\U0001F1F8 \U0001F1F1",
    ":Singapore:": "\U0001F1F8 \U0001F1EC",
    ":Sint_Maarten:": "\U0001F1F8 \U0001F1FD",
    ":Slovakia:": "\U0001F1F8 \U0001F1F0",
    ":Slovenia:": "\U0001F1F8 \U0001F1EE",
    ":Solomon_Islands:": "\U0001F1F8 \U0001F1E7",
    ":Somalia:": "\U0001F1F8 \U0001F1F4",
    ":South_Africa:": "\U0001F1FF \U0001F1E6",
    ":South_Georgia_&_South_Sandwich_Islands:": "\U0001F1EC \U0001F1F8",
    ":South_Korea:": "\U0001F1F0 \U0001F1F7",
    ":South_Sudan:": "\U0001F1F8 \U0001F1F8",
    ":Spain:": "\U0001F1EA \U0001F1F8",
    ":Sri_Lanka:": "\U0001F1F1 \U0001F1F0",
    ":St._Barthélemy:": "\U0001F1E7 \U0001F1F1",
    ":St._Helena:": "\U0001F1F8 \U0001F1ED",
    ":St._Kitts_&_Nevis:": "\U0001F1F0 \U0001F1F3",
    ":St._Lucia:": "\U0001F1F1 \U0001F1E8",
    ":St._Martin:": "\U0001F1F2 \U0001F1EB",
    ":St._Pierre_&_Miquelon:": "\U0001F1F5 \U0001F1F2",
    ":St._Vincent_&_Grenadines:": "\U0001F1FB \U0001F1E8",
    ":Statue_of_Liberty:": "\U0001F5FD",
    ":Sudan:": "\U0001F1F8 \U0001F1E9",
    ":Suriname:": "\U0001F1F8 \U0001F1F7",
    ":Svalbard_&_Jan_Mayen:": "\U0001F1F8 \U0001F1EF",
    ":Swaziland:": "\U0001F1F8 \U0001F1FF",
    ":Sweden:": "\U0001F1F8 \U0001F1EA",
    ":Switzerland:": "\U0001F1E8 \U0001F1ED",
    ":Syria:": "\U0001F1F8 \U0001F1FE",
    ":São_Tomé_&_Príncipe:": "\U0001F1F8 \U0001F1F9",
    ":TOP_arrow:": "\U0001F51D",
    ":Taiwan:": "\U0001F1F9 \U0001F1FC",
    ":Tajikistan:": "\U0001F1F9 \U0001F1EF",
    ":Tanzania:": "\U0001F1F9 \U0001F1FF",
    ":Taurus:": "\U00002649",
    ":Thailand:": "\U0001F1F9 \U0001F1ED",
    ":Timor-Leste:": "\U0001F1F9 \U0001F1F1",
    ":Togo:": "\U0001F1F9 \U0001F1EC",
    ":Tokelau:": "\U0001F1F9 \U0001F1F0",
    ":Tokyo_tower:": "\U0001F5FC",
    ":Tonga:": "\U0001F1F9 \U0001F1F4",
    ":Trinidad_&_Tobago:": "\U0001F1F9 \U0001F1F9",
    ":Tristan_da_Cunha:": "\U0001F1F9 \U0001F1E6",
    ":Tunisia:": "\U0001F1F9 \U0001F1F3",
    ":Turkey:": "\U0001F1F9 \U0001F1F7",
    ":Turkmenistan:": "\U0001F1F9 \U0001F1F2",
    ":Turks_&_Caicos_Islands:": "\U0001F1F9 \U0001F1E8",
    ":Tuvalu:": "\U0001F1F9 \U0001F1FB",
    ":U.S._Outlying_Islands:": "\U0001F1FA \U0001F1F2",
    ":U.S._Virgin_Islands:": "\U0001F1FB \U0001F1EE",
    ":UP!_button:": "\U0001F199",
    ":Uganda:": "\U0001F1FA \U0001F1EC",
    ":Ukraine:": "\U0001F1FA \U0001F1E6",
    ":United_Arab_Emirates:": "\U0001F1E6 \U0001F1EA",
    ":United_Kingdom:": "\U0001F1EC \U0001F1E7",
    ":United_Nations:": "\U0001F1FA \U0001F1F3",
    ":United_States:": "\U0001F1FA \U0001F1F8",
    ":Uruguay:": "\U0001F1FA \U0001F1FE",
    ":Uzbekistan:": "\U0001F1FA \U0001F1FF",
    ":VS_button:": "\U0001F19A",
    ":Vanuatu:": "\U0001F1FB \U0001F1FA",
    ":Vatican_City:": "\U0001F1FB \U0001F1E6",
    ":Venezuela:": "\U0001F1FB \U0001F1EA",
    ":Vietnam:": "\U0001F1FB \U0001F1F3",
    ":Virgo:": "\U0000264D",
    ":Wallis_&_Futuna:": "\U0001F1FC \U0001F1EB",
    ":Western_Sahara:": "\U0001F1EA \U0001F1ED",
    ":Yemen:": "\U0001F1FE \U0001F1EA",
    ":Zambia:": "\U0001F1FF \U0001F1F2",
    ":Zimbabwe:": "\U0001F1FF \U0001F1FC",
    ":admission_tickets:": "\U0001F39F",
    ":aerial_tramway:": "\U0001F6A1",
    ":airplane:": "\U00002708",
    ":airplane_arrival:": "\U0001F6EC",
    ":airplane_departure:": "\U0001F6EB",
    ":alarm_clock:": "\U000023F0",
    ":alembic:": "\U00002697",
    ":alien:": "\U0001F47D",
    ":alien_monster:": "\U0001F47E",
    ":ambulance:": "\U0001F691",
    ":american_football:": "\U0001F3C8",
    ":amphora:": "\U0001F3FA",
    ":anchor:": "\U00002693",
    ":anger_symbol:": "\U0001F4A2",
    ":angry_face:": "\U0001F620",
    ":angry_face_with_horns:": "\U0001F47F",
    ":anguished_face:": "\U0001F627",
    ":ant:": "\U0001F41C",
    ":antenna_bars:": "\U0001F4F6",
    ":anticlockwise_arrows_button:": "\U0001F504",
    ":articulated_lorry:": "\U0001F69B",
    ":artist_palette:": "\U0001F3A8",
    ":astonished_face:": "\U0001F632",
    ":atom_symbol:": "\U0000269B",
    ":automobile:": "\U0001F697",
    ":avocado:": "\U0001F951",
    ":baby:": "\U0001F476",
    ":baby_angel:": "\U0001F47C",
    ":baby_angel_dark_skin_tone:": "\U0001F47C \U0001F3FF",
    ":baby_angel_light_skin_tone:": "\U0001F47C \U0001F3FB",
    ":baby_angel_medium-dark_skin_tone:": "\U0001F47C \U0001F3FE",
    ":baby_angel_medium-light_skin_tone:": "\U0001F47C \U0001F3FC",
    ":baby_angel_medium_skin_tone:": "\U0001F47C \U0001F3FD",
    ":baby_bottle:": "\U0001F37C",
    ":baby_chick:": "\U0001F424",
    ":baby_dark_skin_tone:": "\U0001F476 \U0001F3FF",
    ":baby_light_skin_tone:": "\U0001F476 \U0001F3FB",
    ":baby_medium-dark_skin_tone:": "\U0001F476 \U0001F3FE",
    ":baby_medium-light_skin_tone:": "\U0001F476 \U0001F3FC",
    ":baby_medium_skin_tone:": "\U0001F476 \U0001F3FD",
    ":baby_symbol:": "\U0001F6BC",
    ":backhand_index_pointing_down:": "\U0001F447",
    ":backhand_index_pointing_down_dark_skin_tone:": "\U0001F447 \U0001F3FF",
    ":backhand_index_pointing_down_light_skin_tone:": "\U0001F447 \U0001F3FB",
    ":backhand_index_pointing_down_medium-dark_skin_tone:": "\U0001F447 \U0001F3FE",
    ":backhand_index_pointing_down_medium-light_skin_tone:": "\U0001F447 \U0001F3FC",
    ":backhand_index_pointing_down_medium_skin_tone:": "\U0001F447 \U0001F3FD",
    ":backhand_index_pointing_left:": "\U0001F448",
    ":backhand_index_pointing_left_dark_skin_tone:": "\U0001F448 \U0001F3FF",
    ":backhand_index_pointing_left_light_skin_tone:": "\U0001F448 \U0001F3FB",
    ":backhand_index_pointing_left_medium-dark_skin_tone:": "\U0001F448 \U0001F3FE",
    ":backhand_index_pointing_left_medium-light_skin_tone:": "\U0001F448 \U0001F3FC",
    ":backhand_index_pointing_left_medium_skin_tone:": "\U0001F448 \U0001F3FD",
    ":backhand_index_pointing_right:": "\U0001F449",
    ":backhand_index_pointing_right_dark_skin_tone:": "\U0001F449 \U0001F3FF",
    ":backhand_index_pointing_right_light_skin_tone:": "\U0001F449 \U0001F3FB",
    ":backhand_index_pointing_right_medium-dark_skin_tone:": "\U0001F449 \U0001F3FE",
    ":backhand_index_pointing_right_medium-light_skin_tone:": "\U0001F449 \U0001F3FC",
    ":backhand_index_pointing_right_medium_skin_tone:": "\U0001F449 \U0001F3FD",
    ":backhand_index_pointing_up:": "\U0001F446",
    ":backhand_index_pointing_up_dark_skin_tone:": "\U0001F446 \U0001F3FF",
    ":backhand_index_pointing_up_light_skin_tone:": "\U0001F446 \U0001F3FB",
    ":backhand_index_pointing_up_medium-dark_skin_tone:": "\U0001F446 \U0001F3FE",
    ":backhand_index_pointing_up_medium-light_skin_tone:": "\U0001F446 \U0001F3FC",
    ":backhand_index_pointing_up_medium_skin_tone:": "\U0001F446 \U0001F3FD",
    ":bacon:": "\U0001F953",
    ":badminton:": "\U0001F3F8",
    ":baggage_claim:": "\U0001F6C4",
    ":baguette_bread:": "\U0001F956",
    ":balance_scale:": "\U00002696",
    ":balloon:": "\U0001F388",
    ":ballot_box_with_ballot:": "\U0001F5F3",
    ":ballot_box_with_check:": "\U00002611",
    ":banana:": "\U0001F34C",
    ":bank:": "\U0001F3E6",
    ":bar_chart:": "\U0001F4CA",
    ":barber_pole:": "\U0001F488",
    ":baseball:": "\U000026BE",
    ":basketball:": "\U0001F3C0",
    ":bat:": "\U0001F987",
    ":bathtub:": "\U0001F6C1",
    ":battery:": "\U0001F50B",
    ":beach_with_umbrella:": "\U0001F3D6",
    ":bear_face:": "\U0001F43B",
    ":beating_heart:": "\U0001F493",
    ":bed:": "\U0001F6CF",
    ":beer_mug:": "\U0001F37A",
    ":bell:": "\U0001F514",
    ":bell_with_slash:": "\U0001F515",
    ":bellhop_bell:": "\U0001F6CE",
    ":bento_box:": "\U0001F371",
    ":bicycle:": "\U0001F6B2",
    ":bikini:": "\U0001F459",
    ":biohazard:": "\U00002623",
    ":bird:": "\U0001F426",
    ":birthday_cake:": "\U0001F382",
    ":black_circle:": "\U000026AB",
    ":black_flag:": "\U0001F3F4",
    ":black_heart:": "\U0001F5A4",
    ":black_large_square:": "\U00002B1B",
    ":black_medium-small_square:": "\U000025FE",
    ":black_medium_square:": "\U000025FC",
    ":black_nib:": "\U00002712",
    ":black_small_square:": "\U000025AA",
    ":black_square_button:": "\U0001F532",
    ":blond-haired_man:": "\U0001F471 \U0000200D \U00002642 \U0000FE0F",
    ":blond-haired_man_dark_skin_tone:": "\U0001F471 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":blond-haired_man_light_skin_tone:": "\U0001F471 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":blond-haired_man_medium-dark_skin_tone:": "\U0001F471 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":blond-haired_man_medium-light_skin_tone:": "\U0001F471 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":blond-haired_man_medium_skin_tone:": "\U0001F471 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":blond-haired_person:": "\U0001F471",
    ":blond-haired_person_dark_skin_tone:": "\U0001F471 \U0001F3FF",
    ":blond-haired_person_light_skin_tone:": "\U0001F471 \U0001F3FB",
    ":blond-haired_person_medium-dark_skin_tone:": "\U0001F471 \U0001F3FE",
    ":blond-haired_person_medium-light_skin_tone:": "\U0001F471 \U0001F3FC",
    ":blond-haired_person_medium_skin_tone:": "\U0001F471 \U0001F3FD",
    ":blond-haired_woman:": "\U0001F471 \U0000200D \U00002640 \U0000FE0F",
    ":blond-haired_woman_dark_skin_tone:": "\U0001F471 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":blond-haired_woman_light_skin_tone:": "\U0001F471 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":blond-haired_woman_medium-dark_skin_tone:": "\U0001F471 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":blond-haired_woman_medium-light_skin_tone:": "\U0001F471 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":blond-haired_woman_medium_skin_tone:": "\U0001F471 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":blossom:": "\U0001F33C",
    ":blowfish:": "\U0001F421",
    ":blue_book:": "\U0001F4D8",
    ":blue_circle:": "\U0001F535",
    ":blue_heart:": "\U0001F499",
    ":boar:": "\U0001F417",
    ":bomb:": "\U0001F4A3",
    ":bookmark:": "\U0001F516",
    ":bookmark_tabs:": "\U0001F4D1",
    ":books:": "\U0001F4DA",
    ":bottle_with_popping_cork:": "\U0001F37E",
    ":bouquet:": "\U0001F490",
    ":bow_and_arrow:": "\U0001F3F9",
    ":bowling:": "\U0001F3B3",
    ":boxing_glove:": "\U0001F94A",
    ":boy:": "\U0001F466",
    ":boy_dark_skin_tone:": "\U0001F466 \U0001F3FF",
    ":boy_light_skin_tone:": "\U0001F466 \U0001F3FB",
    ":boy_medium-dark_skin_tone:": "\U0001F466 \U0001F3FE",
    ":boy_medium-light_skin_tone:": "\U0001F466 \U0001F3FC",
    ":boy_medium_skin_tone:": "\U0001F466 \U0001F3FD",
    ":bread:": "\U0001F35E",
    ":bride_with_veil:": "\U0001F470",
    ":bride_with_veil_dark_skin_tone:": "\U0001F470 \U0001F3FF",
    ":bride_with_veil_light_skin_tone:": "\U0001F470 \U0001F3FB",
    ":bride_with_veil_medium-dark_skin_tone:": "\U0001F470 \U0001F3FE",
    ":bride_with_veil_medium-light_skin_tone:": "\U0001F470 \U0001F3FC",
    ":bride_with_veil_medium_skin_tone:": "\U0001F470 \U0001F3FD",
    ":bridge_at_night:": "\U0001F309",
    ":briefcase:": "\U0001F4BC",
    ":bright_button:": "\U0001F506",
    ":broken_heart:": "\U0001F494",
    ":bug:": "\U0001F41B",
    ":building_construction:": "\U0001F3D7",
    ":burrito:": "\U0001F32F",
    ":bus:": "\U0001F68C",
    ":bus_stop:": "\U0001F68F",
    ":bust_in_silhouette:": "\U0001F464",
    ":busts_in_silhouette:": "\U0001F465",
    ":butterfly:": "\U0001F98B",
    ":cactus:": "\U0001F335",
    ":calendar:": "\U0001F4C5",
    ":call_me_hand:": "\U0001F919",
    ":call_me_hand_dark_skin_tone:": "\U0001F919 \U0001F3FF",
    ":call_me_hand_light_skin_tone:": "\U0001F919 \U0001F3FB",
    ":call_me_hand_medium-dark_skin_tone:": "\U0001F919 \U0001F3FE",
    ":call_me_hand_medium-light_skin_tone:": "\U0001F919 \U0001F3FC",
    ":call_me_hand_medium_skin_tone:": "\U0001F919 \U0001F3FD",
    ":camel:": "\U0001F42A",
    ":camera:": "\U0001F4F7",
    ":camera_with_flash:": "\U0001F4F8",
    ":camping:": "\U0001F3D5",
    ":candle:": "\U0001F56F",
    ":candy:": "\U0001F36C",
    ":canoe:": "\U0001F6F6",
    ":card_file_box:": "\U0001F5C3",
    ":card_index:": "\U0001F4C7",
    ":card_index_dividers:": "\U0001F5C2",
    ":carousel_horse:": "\U0001F3A0",
    ":carp_streamer:": "\U0001F38F",
    ":carrot:": "\U0001F955",
    ":castle:": "\U0001F3F0",
    ":cat:": "\U0001F408",
    ":cat_face:": "\U0001F431",
    ":cat_face_with_tears_of_joy:": "\U0001F639",
    ":cat_face_with_wry_smile:": "\U0001F63C",
    ":chains:": "\U000026D3",
    ":chart_decreasing:": "\U0001F4C9",
    ":chart_increasing:": "\U0001F4C8",
    ":chart_increasing_with_yen:": "\U0001F4B9",
    ":cheese_wedge:": "\U0001F9C0",
    ":chequered_flag:": "\U0001F3C1",
    ":cherries:": "\U0001F352",
    ":cherry_blossom:": "\U0001F338",
    ":chestnut:": "\U0001F330",
    ":chicken:": "\U0001F414",
    ":children_crossing:": "\U0001F6B8",
    ":chipmunk:": "\U0001F43F",
    ":chocolate_bar:": "\U0001F36B",
    ":church:": "\U000026EA",
    ":cigarette:": "\U0001F6AC",
    ":cinema:": "\U0001F3A6",
    ":circled_M:": "\U000024C2",
    ":circus_tent:": "\U0001F3AA",
    ":cityscape:": "\U0001F3D9",
    ":cityscape_at_dusk:": "\U0001F306",
    ":clamp:": "\U0001F5DC",
    ":clapper_board:": "\U0001F3AC",
    ":clapping_hands:": "\U0001F44F",
    ":clapping_hands_dark_skin_tone:": "\U0001F44F \U0001F3FF",
    ":clapping_hands_light_skin_tone:": "\U0001F44F \U0001F3FB",
    ":clapping_hands_medium-dark_skin_tone:": "\U0001F44F \U0001F3FE",
    ":clapping_hands_medium-light_skin_tone:": "\U0001F44F \U0001F3FC",
    ":clapping_hands_medium_skin_tone:": "\U0001F44F \U0001F3FD",
    ":classical_building:": "\U0001F3DB",
    ":clinking_beer_mugs:": "\U0001F37B",
    ":clinking_glasses:": "\U0001F942",
    ":clipboard:": "\U0001F4CB",
    ":clockwise_vertical_arrows:": "\U0001F503",
    ":closed_book:": "\U0001F4D5",
    ":closed_mailbox_with_lowered_flag:": "\U0001F4EA",
    ":closed_mailbox_with_raised_flag:": "\U0001F4EB",
    ":closed_umbrella:": "\U0001F302",
    ":cloud:": "\U00002601",
    ":cloud_with_lightning:": "\U0001F329",
    ":cloud_with_lightning_and_rain:": "\U000026C8",
    ":cloud_with_rain:": "\U0001F327",
    ":cloud_with_snow:": "\U0001F328",
    ":clown_face:": "\U0001F921",
    ":club_suit:": "\U00002663",
    ":clutch_bag:": "\U0001F45D",
    ":cocktail_glass:": "\U0001F378",
    ":coffin:": "\U000026B0",
    ":collision:": "\U0001F4A5",
    ":comet:": "\U00002604",
    ":computer_disk:": "\U0001F4BD",
    ":computer_mouse:": "\U0001F5B1",
    ":confetti_ball:": "\U0001F38A",
    ":confounded_face:": "\U0001F616",
    ":confused_face:": "\U0001F615",
    ":construction:": "\U0001F6A7",
    ":construction_worker:": "\U0001F477",
    ":construction_worker_dark_skin_tone:": "\U0001F477 \U0001F3FF",
    ":construction_worker_light_skin_tone:": "\U0001F477 \U0001F3FB",
    ":construction_worker_medium-dark_skin_tone:": "\U0001F477 \U0001F3FE",
    ":construction_worker_medium-light_skin_tone:": "\U0001F477 \U0001F3FC",
    ":construction_worker_medium_skin_tone:": "\U0001F477 \U0001F3FD",
    ":control_knobs:": "\U0001F39B",
    ":convenience_store:": "\U0001F3EA",
    ":cooked_rice:": "\U0001F35A",
    ":cookie:": "\U0001F36A",
    ":cooking:": "\U0001F373",
    ":copyright:": "\U000000A9",
    ":couch_and_lamp:": "\U0001F6CB",
    ":couple_with_heart:": "\U0001F491",
    ":couple_with_heart_man_man:": "\U0001F468 \U0000200D \U00002764 \U0000FE0F \U0000200D \U0001F468",
    ":couple_with_heart_woman_man:": "\U0001F469 \U0000200D \U00002764 \U0000FE0F \U0000200D \U0001F468",
    ":couple_with_heart_woman_woman:": "\U0001F469 \U0000200D \U00002764 \U0000FE0F \U0000200D \U0001F469",
    ":cow:": "\U0001F404",
    ":cow_face:": "\U0001F42E",
    ":cowboy_hat_face:": "\U0001F920",
    ":crab:": "\U0001F980",
    ":crayon:": "\U0001F58D",
    ":credit_card:": "\U0001F4B3",
    ":crescent_moon:": "\U0001F319",
    ":cricket:": "\U0001F3CF",
    ":crocodile:": "\U0001F40A",
    ":croissant:": "\U0001F950",
    ":cross_mark:": "\U0000274C",
    ":cross_mark_button:": "\U0000274E",
    ":crossed_fingers:": "\U0001F91E",
    ":crossed_fingers_dark_skin_tone:": "\U0001F91E \U0001F3FF",
    ":crossed_fingers_light_skin_tone:": "\U0001F91E \U0001F3FB",
    ":crossed_fingers_medium-dark_skin_tone:": "\U0001F91E \U0001F3FE",
    ":crossed_fingers_medium-light_skin_tone:": "\U0001F91E \U0001F3FC",
    ":crossed_fingers_medium_skin_tone:": "\U0001F91E \U0001F3FD",
    ":crossed_flags:": "\U0001F38C",
    ":crossed_swords:": "\U00002694",
    ":crown:": "\U0001F451",
    ":crying_cat_face:": "\U0001F63F",
    ":crying_face:": "\U0001F622",
    ":crystal_ball:": "\U0001F52E",
    ":cucumber:": "\U0001F952",
    ":curly_loop:": "\U000027B0",
    ":currency_exchange:": "\U0001F4B1",
    ":curry_rice:": "\U0001F35B",
    ":custard:": "\U0001F36E",
    ":customs:": "\U0001F6C3",
    ":cyclone:": "\U0001F300",
    ":dagger:": "\U0001F5E1",
    ":dango:": "\U0001F361",
    ":dark_skin_tone:": "\U0001F3FF",
    ":dashing_away:": "\U0001F4A8",
    ":deciduous_tree:": "\U0001F333",
    ":deer:": "\U0001F98C",
    ":delivery_truck:": "\U0001F69A",
    ":department_store:": "\U0001F3EC",
    ":derelict_house:": "\U0001F3DA",
    ":desert:": "\U0001F3DC",
    ":desert_island:": "\U0001F3DD",
    ":desktop_computer:": "\U0001F5A5",
    ":detective:": "\U0001F575",
    ":detective_dark_skin_tone:": "\U0001F575 \U0001F3FF",
    ":detective_light_skin_tone:": "\U0001F575 \U0001F3FB",
    ":detective_medium-dark_skin_tone:": "\U0001F575 \U0001F3FE",
    ":detective_medium-light_skin_tone:": "\U0001F575 \U0001F3FC",
    ":detective_medium_skin_tone:": "\U0001F575 \U0001F3FD",
    ":diamond_suit:": "\U00002666",
    ":diamond_with_a_dot:": "\U0001F4A0",
    ":dim_button:": "\U0001F505",
    ":direct_hit:": "\U0001F3AF",
    ":disappointed_but_relieved_face:": "\U0001F625",
    ":disappointed_face:": "\U0001F61E",
    ":dizzy:": "\U0001F4AB",
    ":dizzy_face:": "\U0001F635",
    ":dog:": "\U0001F415",
    ":dog_face:": "\U0001F436",
    ":dollar_banknote:": "\U0001F4B5",
    ":dolphin:": "\U0001F42C",
    ":door:": "\U0001F6AA",
    ":dotted_six-pointed_star:": "\U0001F52F",
    ":double_curly_loop:": "\U000027BF",
    ":double_exclamation_mark:": "\U0000203C",
    ":doughnut:": "\U0001F369",
    ":dove:": "\U0001F54A",
    ":down-left_arrow:": "\U00002199",
    ":down-right_arrow:": "\U00002198",
    ":down_arrow:": "\U00002B07",
    ":down_button:": "\U0001F53D",
    ":dragon:": "\U0001F409",
    ":dragon_face:": "\U0001F432",
    ":dress:": "\U0001F457",
    ":drooling_face:": "\U0001F924",
    ":droplet:": "\U0001F4A7",
    ":drum:": "\U0001F941",
    ":duck:": "\U0001F986",
    ":dvd:": "\U0001F4C0",
    ":e-mail:": "\U0001F4E7",
    ":eagle:": "\U0001F985",
    ":ear:": "\U0001F442",
    ":ear_dark_skin_tone:": "\U0001F442 \U0001F3FF",
    ":ear_light_skin_tone:": "\U0001F442 \U0001F3FB",
    ":ear_medium-dark_skin_tone:": "\U0001F442 \U0001F3FE",
    ":ear_medium-light_skin_tone:": "\U0001F442 \U0001F3FC",
    ":ear_medium_skin_tone:": "\U0001F442 \U0001F3FD",
    ":ear_of_corn:": "\U0001F33D",
    ":egg:": "\U0001F95A",
    ":eggplant:": "\U0001F346",
    ":eight-pointed_star:": "\U00002734",
    ":eight-spoked_asterisk:": "\U00002733",
    ":eight-thirty:": "\U0001F563",
    ":eight_o’clock:": "\U0001F557",
    ":eject_button:": "\U000023CF",
    ":electric_plug:": "\U0001F50C",
    ":elephant:": "\U0001F418",
    ":eleven-thirty:": "\U0001F566",
    ":eleven_o’clock:": "\U0001F55A",
    ":envelope:": "\U00002709",
    ":envelope_with_arrow:": "\U0001F4E9",
    ":euro_banknote:": "\U0001F4B6",
    ":evergreen_tree:": "\U0001F332",
    ":exclamation_mark:": "\U00002757",
    ":exclamation_question_mark:": "\U00002049",
    ":expressionless_face:": "\U0001F611",
    ":eye:": "\U0001F441",
    ":eye_in_speech_bubble:": "\U0001F441 \U0000FE0F \U0000200D \U0001F5E8 \U0000FE0F",
    ":eyes:": "\U0001F440",
    ":face_blowing_a_kiss:": "\U0001F618",
    ":face_savouring_delicious_food:": "\U0001F60B",
    ":face_screaming_in_fear:": "\U0001F631",
    ":face_with_cold_sweat:": "\U0001F613",
    ":face_with_head-bandage:": "\U0001F915",
    ":face_with_medical_mask:": "\U0001F637",
    ":face_with_open_mouth:": "\U0001F62E",
    ":face_with_open_mouth_&_cold_sweat:": "\U0001F630",
    ":face_with_rolling_eyes:": "\U0001F644",
    ":face_with_steam_from_nose:": "\U0001F624",
    ":face_with_stuck-out_tongue:": "\U0001F61B",
    ":face_with_stuck-out_tongue_&_closed_eyes:": "\U0001F61D",
    ":face_with_stuck-out_tongue_&_winking_eye:": "\U0001F61C",
    ":face_with_tears_of_joy:": "\U0001F602",
    ":face_with_thermometer:": "\U0001F912",
    ":face_without_mouth:": "\U0001F636",
    ":factory:": "\U0001F3ED",
    ":fallen_leaf:": "\U0001F342",
    ":family:": "\U0001F46A",
    ":family_man_boy:": "\U0001F468 \U0000200D \U0001F466",
    ":family_man_boy_boy:": "\U0001F468 \U0000200D \U0001F466 \U0000200D \U0001F466",
    ":family_man_girl:": "\U0001F468 \U0000200D \U0001F467",
    ":family_man_girl_boy:": "\U0001F468 \U0000200D \U0001F467 \U0000200D \U0001F466",
    ":family_man_girl_girl:": "\U0001F468 \U0000200D \U0001F467 \U0000200D \U0001F467",
    ":family_man_man_boy:": "\U0001F468 \U0000200D \U0001F468 \U0000200D \U0001F466",
    ":family_man_man_boy_boy:": "\U0001F468 \U0000200D \U0001F468 \U0000200D \U0001F466 \U0000200D \U0001F466",
    ":family_man_man_girl:": "\U0001F468 \U0000200D \U0001F468 \U0000200D \U0001F467",
    ":family_man_man_girl_boy:": "\U0001F468 \U0000200D \U0001F468 \U0000200D \U0001F467 \U0000200D \U0001F466",
    ":family_man_man_girl_girl:": "\U0001F468 \U0000200D \U0001F468 \U0000200D \U0001F467 \U0000200D \U0001F467",
    ":family_man_woman_boy:": "\U0001F468 \U0000200D \U0001F469 \U0000200D \U0001F466",
    ":family_man_woman_boy_boy:": "\U0001F468 \U0000200D \U0001F469 \U0000200D \U0001F466 \U0000200D \U0001F466",
    ":family_man_woman_girl:": "\U0001F468 \U0000200D \U0001F469 \U0000200D \U0001F467",
    ":family_man_woman_girl_boy:": "\U0001F468 \U0000200D \U0001F469 \U0000200D \U0001F467 \U0000200D \U0001F466",
    ":family_man_woman_girl_girl:": "\U0001F468 \U0000200D \U0001F469 \U0000200D \U0001F467 \U0000200D \U0001F467",
    ":family_woman_boy:": "\U0001F469 \U0000200D \U0001F466",
    ":family_woman_boy_boy:": "\U0001F469 \U0000200D \U0001F466 \U0000200D \U0001F466",
    ":family_woman_girl:": "\U0001F469 \U0000200D \U0001F467",
    ":family_woman_girl_boy:": "\U0001F469 \U0000200D \U0001F467 \U0000200D \U0001F466",
    ":family_woman_girl_girl:": "\U0001F469 \U0000200D \U0001F467 \U0000200D \U0001F467",
    ":family_woman_woman_boy:": "\U0001F469 \U0000200D \U0001F469 \U0000200D \U0001F466",
    ":family_woman_woman_boy_boy:": "\U0001F469 \U0000200D \U0001F469 \U0000200D \U0001F466 \U0000200D \U0001F466",
    ":family_woman_woman_girl:": "\U0001F469 \U0000200D \U0001F469 \U0000200D \U0001F467",
    ":family_woman_woman_girl_boy:": "\U0001F469 \U0000200D \U0001F469 \U0000200D \U0001F467 \U0000200D \U0001F466",
    ":family_woman_woman_girl_girl:": "\U0001F469 \U0000200D \U0001F469 \U0000200D \U0001F467 \U0000200D \U0001F467",
    ":fast-forward_button:": "\U000023E9",
    ":fast_down_button:": "\U000023EC",
    ":fast_reverse_button:": "\U000023EA",
    ":fast_up_button:": "\U000023EB",
    ":fax_machine:": "\U0001F4E0",
    ":fearful_face:": "\U0001F628",
    ":female_sign:": "\U00002640",
    ":ferris_wheel:": "\U0001F3A1",
    ":ferry:": "\U000026F4",
    ":field_hockey:": "\U0001F3D1",
    ":file_cabinet:": "\U0001F5C4",
    ":file_folder:": "\U0001F4C1",
    ":film_frames:": "\U0001F39E",
    ":film_projector:": "\U0001F4FD",
    ":fire:": "\U0001F525",
    ":fire_engine:": "\U0001F692",
    ":fireworks:": "\U0001F386",
    ":first_quarter_moon:": "\U0001F313",
    ":first_quarter_moon_with_face:": "\U0001F31B",
    ":fish:": "\U0001F41F",
    ":fish_cake_with_swirl:": "\U0001F365",
    ":fishing_pole:": "\U0001F3A3",
    ":five-thirty:": "\U0001F560",
    ":five_o’clock:": "\U0001F554",
    ":flag_in_hole:": "\U000026F3",
    ":flashlight:": "\U0001F526",
    ":fleur-de-lis:": "\U0000269C",
    ":flexed_biceps:": "\U0001F4AA",
    ":flexed_biceps_dark_skin_tone:": "\U0001F4AA \U0001F3FF",
    ":flexed_biceps_light_skin_tone:": "\U0001F4AA \U0001F3FB",
    ":flexed_biceps_medium-dark_skin_tone:": "\U0001F4AA \U0001F3FE",
    ":flexed_biceps_medium-light_skin_tone:": "\U0001F4AA \U0001F3FC",
    ":flexed_biceps_medium_skin_tone:": "\U0001F4AA \U0001F3FD",
    ":floppy_disk:": "\U0001F4BE",
    ":flower_playing_cards:": "\U0001F3B4",
    ":flushed_face:": "\U0001F633",
    ":fog:": "\U0001F32B",
    ":foggy:": "\U0001F301",
    ":folded_hands:": "\U0001F64F",
    ":folded_hands_dark_skin_tone:": "\U0001F64F \U0001F3FF",
    ":folded_hands_light_skin_tone:": "\U0001F64F \U0001F3FB",
    ":folded_hands_medium-dark_skin_tone:": "\U0001F64F \U0001F3FE",
    ":folded_hands_medium-light_skin_tone:": "\U0001F64F \U0001F3FC",
    ":folded_hands_medium_skin_tone:": "\U0001F64F \U0001F3FD",
    ":footprints:": "\U0001F463",
    ":fork_and_knife:": "\U0001F374",
    ":fork_and_knife_with_plate:": "\U0001F37D",
    ":fountain:": "\U000026F2",
    ":fountain_pen:": "\U0001F58B",
    ":four-thirty:": "\U0001F55F",
    ":four_leaf_clover:": "\U0001F340",
    ":four_o’clock:": "\U0001F553",
    ":fox_face:": "\U0001F98A",
    ":framed_picture:": "\U0001F5BC",
    ":french_fries:": "\U0001F35F",
    ":fried_shrimp:": "\U0001F364",
    ":frog_face:": "\U0001F438",
    ":front-facing_baby_chick:": "\U0001F425",
    ":frowning_face:": "\U00002639",
    ":frowning_face_with_open_mouth:": "\U0001F626",
    ":fuel_pump:": "\U000026FD",
    ":full_moon:": "\U0001F315",
    ":full_moon_with_face:": "\U0001F31D",
    ":funeral_urn:": "\U000026B1",
    ":game_die:": "\U0001F3B2",
    ":gear:": "\U00002699",
    ":gem_stone:": "\U0001F48E",
    ":ghost:": "\U0001F47B",
    ":girl:": "\U0001F467",
    ":girl_dark_skin_tone:": "\U0001F467 \U0001F3FF",
    ":girl_light_skin_tone:": "\U0001F467 \U0001F3FB",
    ":girl_medium-dark_skin_tone:": "\U0001F467 \U0001F3FE",
    ":girl_medium-light_skin_tone:": "\U0001F467 \U0001F3FC",
    ":girl_medium_skin_tone:": "\U0001F467 \U0001F3FD",
    ":glass_of_milk:": "\U0001F95B",
    ":glasses:": "\U0001F453",
    ":globe_showing_Americas:": "\U0001F30E",
    ":globe_showing_Asia-Australia:": "\U0001F30F",
    ":globe_showing_Europe-Africa:": "\U0001F30D",
    ":globe_with_meridians:": "\U0001F310",
    ":glowing_star:": "\U0001F31F",
    ":goal_net:": "\U0001F945",
    ":goat:": "\U0001F410",
    ":goblin:": "\U0001F47A",
    ":gorilla:": "\U0001F98D",
    ":graduation_cap:": "\U0001F393",
    ":grapes:": "\U0001F347",
    ":green_apple:": "\U0001F34F",
    ":green_book:": "\U0001F4D7",
    ":green_heart:": "\U0001F49A",
    ":green_salad:": "\U0001F957",
    ":grimacing_face:": "\U0001F62C",
    ":grinning_cat_face_with_smiling_eyes:": "\U0001F638",
    ":grinning_face:": "\U0001F600",
    ":grinning_face_with_smiling_eyes:": "\U0001F601",
    ":growing_heart:": "\U0001F497",
    ":guard:": "\U0001F482",
    ":guard_dark_skin_tone:": "\U0001F482 \U0001F3FF",
    ":guard_light_skin_tone:": "\U0001F482 \U0001F3FB",
    ":guard_medium-dark_skin_tone:": "\U0001F482 \U0001F3FE",
    ":guard_medium-light_skin_tone:": "\U0001F482 \U0001F3FC",
    ":guard_medium_skin_tone:": "\U0001F482 \U0001F3FD",
    ":guitar:": "\U0001F3B8",
    ":hamburger:": "\U0001F354",
    ":hammer:": "\U0001F528",
    ":hammer_and_pick:": "\U00002692",
    ":hammer_and_wrench:": "\U0001F6E0",
    ":hamster_face:": "\U0001F439",
    ":handbag:": "\U0001F45C",
    ":handshake:": "\U0001F91D",
    ":hatching_chick:": "\U0001F423",
    ":headphone:": "\U0001F3A7",
    ":hear-no-evil_monkey:": "\U0001F649",
    ":heart_decoration:": "\U0001F49F",
    ":heart_suit:": "\U00002665",
    ":heart_with_arrow:": "\U0001F498",
    ":heart_with_ribbon:": "\U0001F49D",
    ":heavy_check_mark:": "\U00002714",
    ":heavy_division_sign:": "\U00002797",
    ":heavy_dollar_sign:": "\U0001F4B2",
    ":heavy_heart_exclamation:": "\U00002763",
    ":heavy_large_circle:": "\U00002B55",
    ":heavy_minus_sign:": "\U00002796",
    ":heavy_multiplication_x:": "\U00002716",
    ":heavy_plus_sign:": "\U00002795",
    ":helicopter:": "\U0001F681",
    ":herb:": "\U0001F33F",
    ":hibiscus:": "\U0001F33A",
    ":high-heeled_shoe:": "\U0001F460",
    ":high-speed_train:": "\U0001F684",
    ":high-speed_train_with_bullet_nose:": "\U0001F685",
    ":high_voltage:": "\U000026A1",
    ":hole:": "\U0001F573",
    ":honey_pot:": "\U0001F36F",
    ":honeybee:": "\U0001F41D",
    ":horizontal_traffic_light:": "\U0001F6A5",
    ":horse:": "\U0001F40E",
    ":horse_face:": "\U0001F434",
    ":horse_racing:": "\U0001F3C7",
    ":horse_racing_dark_skin_tone:": "\U0001F3C7 \U0001F3FF",
    ":horse_racing_light_skin_tone:": "\U0001F3C7 \U0001F3FB",
    ":horse_racing_medium-dark_skin_tone:": "\U0001F3C7 \U0001F3FE",
    ":horse_racing_medium-light_skin_tone:": "\U0001F3C7 \U0001F3FC",
    ":horse_racing_medium_skin_tone:": "\U0001F3C7 \U0001F3FD",
    ":hospital:": "\U0001F3E5",
    ":hot_beverage:": "\U00002615",
    ":hot_dog:": "\U0001F32D",
    ":hot_pepper:": "\U0001F336",
    ":hot_springs:": "\U00002668",
    ":hotel:": "\U0001F3E8",
    ":hourglass:": "\U0000231B",
    ":hourglass_with_flowing_sand:": "\U000023F3",
    ":house:": "\U0001F3E0",
    ":house_with_garden:": "\U0001F3E1",
    ":hugging_face:": "\U0001F917",
    ":hundred_points:": "\U0001F4AF",
    ":hushed_face:": "\U0001F62F",
    ":ice_cream:": "\U0001F368",
    ":ice_hockey:": "\U0001F3D2",
    ":ice_skate:": "\U000026F8",
    ":inbox_tray:": "\U0001F4E5",
    ":incoming_envelope:": "\U0001F4E8",
    ":index_pointing_up:": "\U0000261D",
    ":index_pointing_up_dark_skin_tone:": "\U0000261D \U0001F3FF",
    ":index_pointing_up_light_skin_tone:": "\U0000261D \U0001F3FB",
    ":index_pointing_up_medium-dark_skin_tone:": "\U0000261D \U0001F3FE",
    ":index_pointing_up_medium-light_skin_tone:": "\U0000261D \U0001F3FC",
    ":index_pointing_up_medium_skin_tone:": "\U0000261D \U0001F3FD",
    ":information:": "\U00002139",
    ":input_latin_letters:": "\U0001F524",
    ":input_latin_lowercase:": "\U0001F521",
    ":input_latin_uppercase:": "\U0001F520",
    ":input_numbers:": "\U0001F522",
    ":input_symbols:": "\U0001F523",
    ":jack-o-lantern:": "\U0001F383",
    ":jeans:": "\U0001F456",
    ":joker:": "\U0001F0CF",
    ":joystick:": "\U0001F579",
    ":kaaba:": "\U0001F54B",
    ":key:": "\U0001F511",
    ":keyboard:": "\U00002328",
    ":keycap_#:": "\U00000023 \U0000FE0F \U000020E3",
    # u':keycap_*:': u'\U0000002A \U0000FE0F \U000020E3',
    ":keycap_0:": "\U00000030 \U0000FE0F \U000020E3",
    ":keycap_1:": "\U00000031 \U0000FE0F \U000020E3",
    ":keycap_10:": "\U0001F51F",
    ":keycap_2:": "\U00000032 \U0000FE0F \U000020E3",
    ":keycap_3:": "\U00000033 \U0000FE0F \U000020E3",
    ":keycap_4:": "\U00000034 \U0000FE0F \U000020E3",
    ":keycap_5:": "\U00000035 \U0000FE0F \U000020E3",
    ":keycap_6:": "\U00000036 \U0000FE0F \U000020E3",
    ":keycap_7:": "\U00000037 \U0000FE0F \U000020E3",
    ":keycap_8:": "\U00000038 \U0000FE0F \U000020E3",
    ":keycap_9:": "\U00000039 \U0000FE0F \U000020E3",
    ":kick_scooter:": "\U0001F6F4",
    ":kimono:": "\U0001F458",
    ":kiss:": "\U0001F48F",
    ":kiss_man_man:": "\U0001F468 \U0000200D \U00002764 \U0000FE0F \U0000200D \U0001F48B \U0000200D \U0001F468",
    ":kiss_mark:": "\U0001F48B",
    ":kiss_woman_man:": "\U0001F469 \U0000200D \U00002764 \U0000FE0F \U0000200D \U0001F48B \U0000200D \U0001F468",
    ":kiss_woman_woman:": "\U0001F469 \U0000200D \U00002764 \U0000FE0F \U0000200D \U0001F48B \U0000200D \U0001F469",
    ":kissing_cat_face_with_closed_eyes:": "\U0001F63D",
    ":kissing_face:": "\U0001F617",
    ":kissing_face_with_closed_eyes:": "\U0001F61A",
    ":kissing_face_with_smiling_eyes:": "\U0001F619",
    ":kitchen_knife:": "\U0001F52A",
    ":kiwi_fruit:": "\U0001F95D",
    ":koala:": "\U0001F428",
    ":label:": "\U0001F3F7",
    ":lady_beetle:": "\U0001F41E",
    ":laptop_computer:": "\U0001F4BB",
    ":large_blue_diamond:": "\U0001F537",
    ":large_orange_diamond:": "\U0001F536",
    ":last_quarter_moon:": "\U0001F317",
    ":last_quarter_moon_with_face:": "\U0001F31C",
    ":last_track_button:": "\U000023EE",
    ":latin_cross:": "\U0000271D",
    ":leaf_fluttering_in_wind:": "\U0001F343",
    ":ledger:": "\U0001F4D2",
    ":left-facing_fist:": "\U0001F91B",
    ":left-facing_fist_dark_skin_tone:": "\U0001F91B \U0001F3FF",
    ":left-facing_fist_light_skin_tone:": "\U0001F91B \U0001F3FB",
    ":left-facing_fist_medium-dark_skin_tone:": "\U0001F91B \U0001F3FE",
    ":left-facing_fist_medium-light_skin_tone:": "\U0001F91B \U0001F3FC",
    ":left-facing_fist_medium_skin_tone:": "\U0001F91B \U0001F3FD",
    ":left-pointing_magnifying_glass:": "\U0001F50D",
    ":left-right_arrow:": "\U00002194",
    ":left_arrow:": "\U00002B05",
    ":left_arrow_curving_right:": "\U000021AA",
    ":left_luggage:": "\U0001F6C5",
    ":left_speech_bubble:": "\U0001F5E8",
    ":lemon:": "\U0001F34B",
    ":leopard:": "\U0001F406",
    ":level_slider:": "\U0001F39A",
    ":light_bulb:": "\U0001F4A1",
    ":light_rail:": "\U0001F688",
    ":light_skin_tone:": "\U0001F3FB",
    ":link:": "\U0001F517",
    ":linked_paperclips:": "\U0001F587",
    ":lion_face:": "\U0001F981",
    ":lipstick:": "\U0001F484",
    ":litter_in_bin_sign:": "\U0001F6AE",
    ":lizard:": "\U0001F98E",
    ":locked:": "\U0001F512",
    ":locked_with_key:": "\U0001F510",
    ":locked_with_pen:": "\U0001F50F",
    ":locomotive:": "\U0001F682",
    ":lollipop:": "\U0001F36D",
    ":loudly_crying_face:": "\U0001F62D",
    ":loudspeaker:": "\U0001F4E2",
    ":love_hotel:": "\U0001F3E9",
    ":love_letter:": "\U0001F48C",
    ":lying_face:": "\U0001F925",
    ":mahjong_red_dragon:": "\U0001F004",
    ":male_sign:": "\U00002642",
    ":man:": "\U0001F468",
    ":man_and_woman_holding_hands:": "\U0001F46B",
    ":man_artist:": "\U0001F468 \U0000200D \U0001F3A8",
    ":man_artist_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F3A8",
    ":man_artist_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F3A8",
    ":man_artist_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F3A8",
    ":man_artist_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F3A8",
    ":man_artist_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F3A8",
    ":man_astronaut:": "\U0001F468 \U0000200D \U0001F680",
    ":man_astronaut_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F680",
    ":man_astronaut_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F680",
    ":man_astronaut_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F680",
    ":man_astronaut_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F680",
    ":man_astronaut_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F680",
    ":man_biking:": "\U0001F6B4 \U0000200D \U00002642 \U0000FE0F",
    ":man_biking_dark_skin_tone:": "\U0001F6B4 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_biking_light_skin_tone:": "\U0001F6B4 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_biking_medium-dark_skin_tone:": "\U0001F6B4 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_biking_medium-light_skin_tone:": "\U0001F6B4 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_biking_medium_skin_tone:": "\U0001F6B4 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_bouncing_ball:": "\U000026F9 \U0000FE0F \U0000200D \U00002642 \U0000FE0F",
    ":man_bouncing_ball_dark_skin_tone:": "\U000026F9 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_bouncing_ball_light_skin_tone:": "\U000026F9 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_bouncing_ball_medium-dark_skin_tone:": "\U000026F9 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_bouncing_ball_medium-light_skin_tone:": "\U000026F9 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_bouncing_ball_medium_skin_tone:": "\U000026F9 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_bowing:": "\U0001F647 \U0000200D \U00002642 \U0000FE0F",
    ":man_bowing_dark_skin_tone:": "\U0001F647 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_bowing_light_skin_tone:": "\U0001F647 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_bowing_medium-dark_skin_tone:": "\U0001F647 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_bowing_medium-light_skin_tone:": "\U0001F647 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_bowing_medium_skin_tone:": "\U0001F647 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_cartwheeling:": "\U0001F938 \U0000200D \U00002642 \U0000FE0F",
    ":man_cartwheeling_dark_skin_tone:": "\U0001F938 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_cartwheeling_light_skin_tone:": "\U0001F938 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_cartwheeling_medium-dark_skin_tone:": "\U0001F938 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_cartwheeling_medium-light_skin_tone:": "\U0001F938 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_cartwheeling_medium_skin_tone:": "\U0001F938 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_construction_worker:": "\U0001F477 \U0000200D \U00002642 \U0000FE0F",
    ":man_construction_worker_dark_skin_tone:": "\U0001F477 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_construction_worker_light_skin_tone:": "\U0001F477 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_construction_worker_medium-dark_skin_tone:": "\U0001F477 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_construction_worker_medium-light_skin_tone:": "\U0001F477 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_construction_worker_medium_skin_tone:": "\U0001F477 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_cook:": "\U0001F468 \U0000200D \U0001F373",
    ":man_cook_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F373",
    ":man_cook_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F373",
    ":man_cook_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F373",
    ":man_cook_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F373",
    ":man_cook_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F373",
    ":man_dancing:": "\U0001F57A",
    ":man_dancing_dark_skin_tone:": "\U0001F57A \U0001F3FF",
    ":man_dancing_light_skin_tone:": "\U0001F57A \U0001F3FB",
    ":man_dancing_medium-dark_skin_tone:": "\U0001F57A \U0001F3FE",
    ":man_dancing_medium-light_skin_tone:": "\U0001F57A \U0001F3FC",
    ":man_dancing_medium_skin_tone:": "\U0001F57A \U0001F3FD",
    ":man_dark_skin_tone:": "\U0001F468 \U0001F3FF",
    ":man_detective:": "\U0001F575 \U0000FE0F \U0000200D \U00002642 \U0000FE0F",
    ":man_detective_dark_skin_tone:": "\U0001F575 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_detective_light_skin_tone:": "\U0001F575 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_detective_medium-dark_skin_tone:": "\U0001F575 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_detective_medium-light_skin_tone:": "\U0001F575 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_detective_medium_skin_tone:": "\U0001F575 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_facepalming:": "\U0001F926 \U0000200D \U00002642 \U0000FE0F",
    ":man_facepalming_dark_skin_tone:": "\U0001F926 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_facepalming_light_skin_tone:": "\U0001F926 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_facepalming_medium-dark_skin_tone:": "\U0001F926 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_facepalming_medium-light_skin_tone:": "\U0001F926 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_facepalming_medium_skin_tone:": "\U0001F926 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_factory_worker:": "\U0001F468 \U0000200D \U0001F3ED",
    ":man_factory_worker_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F3ED",
    ":man_factory_worker_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F3ED",
    ":man_factory_worker_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F3ED",
    ":man_factory_worker_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F3ED",
    ":man_factory_worker_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F3ED",
    ":man_farmer:": "\U0001F468 \U0000200D \U0001F33E",
    ":man_farmer_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F33E",
    ":man_farmer_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F33E",
    ":man_farmer_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F33E",
    ":man_farmer_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F33E",
    ":man_farmer_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F33E",
    ":man_firefighter:": "\U0001F468 \U0000200D \U0001F692",
    ":man_firefighter_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F692",
    ":man_firefighter_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F692",
    ":man_firefighter_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F692",
    ":man_firefighter_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F692",
    ":man_firefighter_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F692",
    ":man_frowning:": "\U0001F64D \U0000200D \U00002642 \U0000FE0F",
    ":man_frowning_dark_skin_tone:": "\U0001F64D \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_frowning_light_skin_tone:": "\U0001F64D \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_frowning_medium-dark_skin_tone:": "\U0001F64D \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_frowning_medium-light_skin_tone:": "\U0001F64D \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_frowning_medium_skin_tone:": "\U0001F64D \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_NO:": "\U0001F645 \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_NO_dark_skin_tone:": "\U0001F645 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_NO_light_skin_tone:": "\U0001F645 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_NO_medium-dark_skin_tone:": "\U0001F645 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_NO_medium-light_skin_tone:": "\U0001F645 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_NO_medium_skin_tone:": "\U0001F645 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_OK:": "\U0001F646 \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_OK_dark_skin_tone:": "\U0001F646 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_OK_light_skin_tone:": "\U0001F646 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_OK_medium-dark_skin_tone:": "\U0001F646 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_OK_medium-light_skin_tone:": "\U0001F646 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_gesturing_OK_medium_skin_tone:": "\U0001F646 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_haircut:": "\U0001F487 \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_haircut_dark_skin_tone:": "\U0001F487 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_haircut_light_skin_tone:": "\U0001F487 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_haircut_medium-dark_skin_tone:": "\U0001F487 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_haircut_medium-light_skin_tone:": "\U0001F487 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_haircut_medium_skin_tone:": "\U0001F487 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_massage:": "\U0001F486 \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_massage_dark_skin_tone:": "\U0001F486 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_massage_light_skin_tone:": "\U0001F486 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_massage_medium-dark_skin_tone:": "\U0001F486 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_massage_medium-light_skin_tone:": "\U0001F486 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_getting_massage_medium_skin_tone:": "\U0001F486 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_golfing:": "\U0001F3CC \U0000FE0F \U0000200D \U00002642 \U0000FE0F",
    ":man_golfing_dark_skin_tone:": "\U0001F3CC \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_golfing_light_skin_tone:": "\U0001F3CC \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_golfing_medium-dark_skin_tone:": "\U0001F3CC \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_golfing_medium-light_skin_tone:": "\U0001F3CC \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_golfing_medium_skin_tone:": "\U0001F3CC \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_guard:": "\U0001F482 \U0000200D \U00002642 \U0000FE0F",
    ":man_guard_dark_skin_tone:": "\U0001F482 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_guard_light_skin_tone:": "\U0001F482 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_guard_medium-dark_skin_tone:": "\U0001F482 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_guard_medium-light_skin_tone:": "\U0001F482 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_guard_medium_skin_tone:": "\U0001F482 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_health_worker:": "\U0001F468 \U0000200D \U00002695 \U0000FE0F",
    ":man_health_worker_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U00002695 \U0000FE0F",
    ":man_health_worker_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U00002695 \U0000FE0F",
    ":man_health_worker_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U00002695 \U0000FE0F",
    ":man_health_worker_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U00002695 \U0000FE0F",
    ":man_health_worker_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U00002695 \U0000FE0F",
    ":man_in_business_suit_levitating:": "\U0001F574",
    ":man_in_business_suit_levitating_dark_skin_tone:": "\U0001F574 \U0001F3FF",
    ":man_in_business_suit_levitating_light_skin_tone:": "\U0001F574 \U0001F3FB",
    ":man_in_business_suit_levitating_medium-dark_skin_tone:": "\U0001F574 \U0001F3FE",
    ":man_in_business_suit_levitating_medium-light_skin_tone:": "\U0001F574 \U0001F3FC",
    ":man_in_business_suit_levitating_medium_skin_tone:": "\U0001F574 \U0001F3FD",
    ":man_in_tuxedo:": "\U0001F935",
    ":man_in_tuxedo_dark_skin_tone:": "\U0001F935 \U0001F3FF",
    ":man_in_tuxedo_light_skin_tone:": "\U0001F935 \U0001F3FB",
    ":man_in_tuxedo_medium-dark_skin_tone:": "\U0001F935 \U0001F3FE",
    ":man_in_tuxedo_medium-light_skin_tone:": "\U0001F935 \U0001F3FC",
    ":man_in_tuxedo_medium_skin_tone:": "\U0001F935 \U0001F3FD",
    ":man_judge:": "\U0001F468 \U0000200D \U00002696 \U0000FE0F",
    ":man_judge_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U00002696 \U0000FE0F",
    ":man_judge_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U00002696 \U0000FE0F",
    ":man_judge_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U00002696 \U0000FE0F",
    ":man_judge_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U00002696 \U0000FE0F",
    ":man_judge_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U00002696 \U0000FE0F",
    ":man_juggling:": "\U0001F939 \U0000200D \U00002642 \U0000FE0F",
    ":man_juggling_dark_skin_tone:": "\U0001F939 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_juggling_light_skin_tone:": "\U0001F939 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_juggling_medium-dark_skin_tone:": "\U0001F939 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_juggling_medium-light_skin_tone:": "\U0001F939 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_juggling_medium_skin_tone:": "\U0001F939 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_lifting_weights:": "\U0001F3CB \U0000FE0F \U0000200D \U00002642 \U0000FE0F",
    ":man_lifting_weights_dark_skin_tone:": "\U0001F3CB \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_lifting_weights_light_skin_tone:": "\U0001F3CB \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_lifting_weights_medium-dark_skin_tone:": "\U0001F3CB \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_lifting_weights_medium-light_skin_tone:": "\U0001F3CB \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_lifting_weights_medium_skin_tone:": "\U0001F3CB \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_light_skin_tone:": "\U0001F468 \U0001F3FB",
    ":man_mechanic:": "\U0001F468 \U0000200D \U0001F527",
    ":man_mechanic_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F527",
    ":man_mechanic_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F527",
    ":man_mechanic_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F527",
    ":man_mechanic_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F527",
    ":man_mechanic_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F527",
    ":man_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE",
    ":man_medium-light_skin_tone:": "\U0001F468 \U0001F3FC",
    ":man_medium_skin_tone:": "\U0001F468 \U0001F3FD",
    ":man_mountain_biking:": "\U0001F6B5 \U0000200D \U00002642 \U0000FE0F",
    ":man_mountain_biking_dark_skin_tone:": "\U0001F6B5 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_mountain_biking_light_skin_tone:": "\U0001F6B5 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_mountain_biking_medium-dark_skin_tone:": "\U0001F6B5 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_mountain_biking_medium-light_skin_tone:": "\U0001F6B5 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_mountain_biking_medium_skin_tone:": "\U0001F6B5 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_office_worker:": "\U0001F468 \U0000200D \U0001F4BC",
    ":man_office_worker_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F4BC",
    ":man_office_worker_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F4BC",
    ":man_office_worker_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F4BC",
    ":man_office_worker_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F4BC",
    ":man_office_worker_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F4BC",
    ":man_pilot:": "\U0001F468 \U0000200D \U00002708 \U0000FE0F",
    ":man_pilot_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U00002708 \U0000FE0F",
    ":man_pilot_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U00002708 \U0000FE0F",
    ":man_pilot_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U00002708 \U0000FE0F",
    ":man_pilot_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U00002708 \U0000FE0F",
    ":man_pilot_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U00002708 \U0000FE0F",
    ":man_playing_handball:": "\U0001F93E \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_handball_dark_skin_tone:": "\U0001F93E \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_handball_light_skin_tone:": "\U0001F93E \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_handball_medium-dark_skin_tone:": "\U0001F93E \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_handball_medium-light_skin_tone:": "\U0001F93E \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_handball_medium_skin_tone:": "\U0001F93E \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_water_polo:": "\U0001F93D \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_water_polo_dark_skin_tone:": "\U0001F93D \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_water_polo_light_skin_tone:": "\U0001F93D \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_water_polo_medium-dark_skin_tone:": "\U0001F93D \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_water_polo_medium-light_skin_tone:": "\U0001F93D \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_playing_water_polo_medium_skin_tone:": "\U0001F93D \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_police_officer:": "\U0001F46E \U0000200D \U00002642 \U0000FE0F",
    ":man_police_officer_dark_skin_tone:": "\U0001F46E \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_police_officer_light_skin_tone:": "\U0001F46E \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_police_officer_medium-dark_skin_tone:": "\U0001F46E \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_police_officer_medium-light_skin_tone:": "\U0001F46E \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_police_officer_medium_skin_tone:": "\U0001F46E \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_pouting:": "\U0001F64E \U0000200D \U00002642 \U0000FE0F",
    ":man_pouting_dark_skin_tone:": "\U0001F64E \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_pouting_light_skin_tone:": "\U0001F64E \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_pouting_medium-dark_skin_tone:": "\U0001F64E \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_pouting_medium-light_skin_tone:": "\U0001F64E \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_pouting_medium_skin_tone:": "\U0001F64E \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_raising_hand:": "\U0001F64B \U0000200D \U00002642 \U0000FE0F",
    ":man_raising_hand_dark_skin_tone:": "\U0001F64B \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_raising_hand_light_skin_tone:": "\U0001F64B \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_raising_hand_medium-dark_skin_tone:": "\U0001F64B \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_raising_hand_medium-light_skin_tone:": "\U0001F64B \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_raising_hand_medium_skin_tone:": "\U0001F64B \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_rowing_boat:": "\U0001F6A3 \U0000200D \U00002642 \U0000FE0F",
    ":man_rowing_boat_dark_skin_tone:": "\U0001F6A3 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_rowing_boat_light_skin_tone:": "\U0001F6A3 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_rowing_boat_medium-dark_skin_tone:": "\U0001F6A3 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_rowing_boat_medium-light_skin_tone:": "\U0001F6A3 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_rowing_boat_medium_skin_tone:": "\U0001F6A3 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_running:": "\U0001F3C3 \U0000200D \U00002642 \U0000FE0F",
    ":man_running_dark_skin_tone:": "\U0001F3C3 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_running_light_skin_tone:": "\U0001F3C3 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_running_medium-dark_skin_tone:": "\U0001F3C3 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_running_medium-light_skin_tone:": "\U0001F3C3 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_running_medium_skin_tone:": "\U0001F3C3 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_scientist:": "\U0001F468 \U0000200D \U0001F52C",
    ":man_scientist_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F52C",
    ":man_scientist_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F52C",
    ":man_scientist_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F52C",
    ":man_scientist_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F52C",
    ":man_scientist_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F52C",
    ":man_shrugging:": "\U0001F937 \U0000200D \U00002642 \U0000FE0F",
    ":man_shrugging_dark_skin_tone:": "\U0001F937 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_shrugging_light_skin_tone:": "\U0001F937 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_shrugging_medium-dark_skin_tone:": "\U0001F937 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_shrugging_medium-light_skin_tone:": "\U0001F937 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_shrugging_medium_skin_tone:": "\U0001F937 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_singer:": "\U0001F468 \U0000200D \U0001F3A4",
    ":man_singer_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F3A4",
    ":man_singer_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F3A4",
    ":man_singer_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F3A4",
    ":man_singer_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F3A4",
    ":man_singer_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F3A4",
    ":man_student:": "\U0001F468 \U0000200D \U0001F393",
    ":man_student_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F393",
    ":man_student_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F393",
    ":man_student_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F393",
    ":man_student_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F393",
    ":man_student_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F393",
    ":man_surfing:": "\U0001F3C4 \U0000200D \U00002642 \U0000FE0F",
    ":man_surfing_dark_skin_tone:": "\U0001F3C4 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_surfing_light_skin_tone:": "\U0001F3C4 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_surfing_medium-dark_skin_tone:": "\U0001F3C4 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_surfing_medium-light_skin_tone:": "\U0001F3C4 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_surfing_medium_skin_tone:": "\U0001F3C4 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_swimming:": "\U0001F3CA \U0000200D \U00002642 \U0000FE0F",
    ":man_swimming_dark_skin_tone:": "\U0001F3CA \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_swimming_light_skin_tone:": "\U0001F3CA \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_swimming_medium-dark_skin_tone:": "\U0001F3CA \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_swimming_medium-light_skin_tone:": "\U0001F3CA \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_swimming_medium_skin_tone:": "\U0001F3CA \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_teacher:": "\U0001F468 \U0000200D \U0001F3EB",
    ":man_teacher_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F3EB",
    ":man_teacher_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F3EB",
    ":man_teacher_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F3EB",
    ":man_teacher_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F3EB",
    ":man_teacher_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F3EB",
    ":man_technologist:": "\U0001F468 \U0000200D \U0001F4BB",
    ":man_technologist_dark_skin_tone:": "\U0001F468 \U0001F3FF \U0000200D \U0001F4BB",
    ":man_technologist_light_skin_tone:": "\U0001F468 \U0001F3FB \U0000200D \U0001F4BB",
    ":man_technologist_medium-dark_skin_tone:": "\U0001F468 \U0001F3FE \U0000200D \U0001F4BB",
    ":man_technologist_medium-light_skin_tone:": "\U0001F468 \U0001F3FC \U0000200D \U0001F4BB",
    ":man_technologist_medium_skin_tone:": "\U0001F468 \U0001F3FD \U0000200D \U0001F4BB",
    ":man_tipping_hand:": "\U0001F481 \U0000200D \U00002642 \U0000FE0F",
    ":man_tipping_hand_dark_skin_tone:": "\U0001F481 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_tipping_hand_light_skin_tone:": "\U0001F481 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_tipping_hand_medium-dark_skin_tone:": "\U0001F481 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_tipping_hand_medium-light_skin_tone:": "\U0001F481 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_tipping_hand_medium_skin_tone:": "\U0001F481 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_walking:": "\U0001F6B6 \U0000200D \U00002642 \U0000FE0F",
    ":man_walking_dark_skin_tone:": "\U0001F6B6 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_walking_light_skin_tone:": "\U0001F6B6 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_walking_medium-dark_skin_tone:": "\U0001F6B6 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_walking_medium-light_skin_tone:": "\U0001F6B6 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_walking_medium_skin_tone:": "\U0001F6B6 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_wearing_turban:": "\U0001F473 \U0000200D \U00002642 \U0000FE0F",
    ":man_wearing_turban_dark_skin_tone:": "\U0001F473 \U0001F3FF \U0000200D \U00002642 \U0000FE0F",
    ":man_wearing_turban_light_skin_tone:": "\U0001F473 \U0001F3FB \U0000200D \U00002642 \U0000FE0F",
    ":man_wearing_turban_medium-dark_skin_tone:": "\U0001F473 \U0001F3FE \U0000200D \U00002642 \U0000FE0F",
    ":man_wearing_turban_medium-light_skin_tone:": "\U0001F473 \U0001F3FC \U0000200D \U00002642 \U0000FE0F",
    ":man_wearing_turban_medium_skin_tone:": "\U0001F473 \U0001F3FD \U0000200D \U00002642 \U0000FE0F",
    ":man_with_Chinese_cap:": "\U0001F472",
    ":man_with_Chinese_cap_dark_skin_tone:": "\U0001F472 \U0001F3FF",
    ":man_with_Chinese_cap_light_skin_tone:": "\U0001F472 \U0001F3FB",
    ":man_with_Chinese_cap_medium-dark_skin_tone:": "\U0001F472 \U0001F3FE",
    ":man_with_Chinese_cap_medium-light_skin_tone:": "\U0001F472 \U0001F3FC",
    ":man_with_Chinese_cap_medium_skin_tone:": "\U0001F472 \U0001F3FD",
    ":mantelpiece_clock:": "\U0001F570",
    ":man’s_shoe:": "\U0001F45E",
    ":map_of_Japan:": "\U0001F5FE",
    ":maple_leaf:": "\U0001F341",
    ":martial_arts_uniform:": "\U0001F94B",
    ":meat_on_bone:": "\U0001F356",
    ":medical_symbol:": "\U00002695",
    ":medium-dark_skin_tone:": "\U0001F3FE",
    ":medium-light_skin_tone:": "\U0001F3FC",
    ":medium_skin_tone:": "\U0001F3FD",
    ":megaphone:": "\U0001F4E3",
    ":melon:": "\U0001F348",
    ":memo:": "\U0001F4DD",
    ":men_with_bunny_ears_partying:": "\U0001F46F \U0000200D \U00002642 \U0000FE0F",
    ":men_wrestling:": "\U0001F93C \U0000200D \U00002642 \U0000FE0F",
    ":menorah:": "\U0001F54E",
    ":men’s_room:": "\U0001F6B9",
    ":metro:": "\U0001F687",
    ":microphone:": "\U0001F3A4",
    ":microscope:": "\U0001F52C",
    ":middle_finger:": "\U0001F595",
    ":middle_finger_dark_skin_tone:": "\U0001F595 \U0001F3FF",
    ":middle_finger_light_skin_tone:": "\U0001F595 \U0001F3FB",
    ":middle_finger_medium-dark_skin_tone:": "\U0001F595 \U0001F3FE",
    ":middle_finger_medium-light_skin_tone:": "\U0001F595 \U0001F3FC",
    ":middle_finger_medium_skin_tone:": "\U0001F595 \U0001F3FD",
    ":military_medal:": "\U0001F396",
    ":milky_way:": "\U0001F30C",
    ":minibus:": "\U0001F690",
    ":moai:": "\U0001F5FF",
    ":mobile_phone:": "\U0001F4F1",
    ":mobile_phone_off:": "\U0001F4F4",
    ":mobile_phone_with_arrow:": "\U0001F4F2",
    ":money-mouth_face:": "\U0001F911",
    ":money_bag:": "\U0001F4B0",
    ":money_with_wings:": "\U0001F4B8",
    ":monkey:": "\U0001F412",
    ":monkey_face:": "\U0001F435",
    ":monorail:": "\U0001F69D",
    ":moon_viewing_ceremony:": "\U0001F391",
    ":mosque:": "\U0001F54C",
    ":motor_boat:": "\U0001F6E5",
    ":motor_scooter:": "\U0001F6F5",
    ":motorcycle:": "\U0001F3CD",
    ":motorway:": "\U0001F6E3",
    ":mount_fuji:": "\U0001F5FB",
    ":mountain:": "\U000026F0",
    ":mountain_cableway:": "\U0001F6A0",
    ":mountain_railway:": "\U0001F69E",
    ":mouse:": "\U0001F401",
    ":mouse_face:": "\U0001F42D",
    ":mouth:": "\U0001F444",
    ":movie_camera:": "\U0001F3A5",
    ":mushroom:": "\U0001F344",
    ":musical_keyboard:": "\U0001F3B9",
    ":musical_note:": "\U0001F3B5",
    ":musical_notes:": "\U0001F3B6",
    ":musical_score:": "\U0001F3BC",
    ":muted_speaker:": "\U0001F507",
    ":nail_polish:": "\U0001F485",
    ":nail_polish_dark_skin_tone:": "\U0001F485 \U0001F3FF",
    ":nail_polish_light_skin_tone:": "\U0001F485 \U0001F3FB",
    ":nail_polish_medium-dark_skin_tone:": "\U0001F485 \U0001F3FE",
    ":nail_polish_medium-light_skin_tone:": "\U0001F485 \U0001F3FC",
    ":nail_polish_medium_skin_tone:": "\U0001F485 \U0001F3FD",
    ":name_badge:": "\U0001F4DB",
    ":national_park:": "\U0001F3DE",
    ":nauseated_face:": "\U0001F922",
    ":necktie:": "\U0001F454",
    ":nerd_face:": "\U0001F913",
    ":neutral_face:": "\U0001F610",
    ":new_moon:": "\U0001F311",
    ":new_moon_face:": "\U0001F31A",
    ":newspaper:": "\U0001F4F0",
    ":next_track_button:": "\U000023ED",
    ":night_with_stars:": "\U0001F303",
    ":nine-thirty:": "\U0001F564",
    ":nine_o’clock:": "\U0001F558",
    ":no_bicycles:": "\U0001F6B3",
    ":no_entry:": "\U000026D4",
    ":no_littering:": "\U0001F6AF",
    ":no_mobile_phones:": "\U0001F4F5",
    ":no_one_under_eighteen:": "\U0001F51E",
    ":no_pedestrians:": "\U0001F6B7",
    ":no_smoking:": "\U0001F6AD",
    ":non-potable_water:": "\U0001F6B1",
    ":nose:": "\U0001F443",
    ":nose_dark_skin_tone:": "\U0001F443 \U0001F3FF",
    ":nose_light_skin_tone:": "\U0001F443 \U0001F3FB",
    ":nose_medium-dark_skin_tone:": "\U0001F443 \U0001F3FE",
    ":nose_medium-light_skin_tone:": "\U0001F443 \U0001F3FC",
    ":nose_medium_skin_tone:": "\U0001F443 \U0001F3FD",
    ":notebook:": "\U0001F4D3",
    ":notebook_with_decorative_cover:": "\U0001F4D4",
    ":nut_and_bolt:": "\U0001F529",
    ":octopus:": "\U0001F419",
    ":oden:": "\U0001F362",
    ":office_building:": "\U0001F3E2",
    ":ogre:": "\U0001F479",
    ":oil_drum:": "\U0001F6E2",
    ":old_key:": "\U0001F5DD",
    ":old_man:": "\U0001F474",
    ":old_man_dark_skin_tone:": "\U0001F474 \U0001F3FF",
    ":old_man_light_skin_tone:": "\U0001F474 \U0001F3FB",
    ":old_man_medium-dark_skin_tone:": "\U0001F474 \U0001F3FE",
    ":old_man_medium-light_skin_tone:": "\U0001F474 \U0001F3FC",
    ":old_man_medium_skin_tone:": "\U0001F474 \U0001F3FD",
    ":old_woman:": "\U0001F475",
    ":old_woman_dark_skin_tone:": "\U0001F475 \U0001F3FF",
    ":old_woman_light_skin_tone:": "\U0001F475 \U0001F3FB",
    ":old_woman_medium-dark_skin_tone:": "\U0001F475 \U0001F3FE",
    ":old_woman_medium-light_skin_tone:": "\U0001F475 \U0001F3FC",
    ":old_woman_medium_skin_tone:": "\U0001F475 \U0001F3FD",
    ":om:": "\U0001F549",
    ":oncoming_automobile:": "\U0001F698",
    ":oncoming_bus:": "\U0001F68D",
    ":oncoming_fist:": "\U0001F44A",
    ":oncoming_fist_dark_skin_tone:": "\U0001F44A \U0001F3FF",
    ":oncoming_fist_light_skin_tone:": "\U0001F44A \U0001F3FB",
    ":oncoming_fist_medium-dark_skin_tone:": "\U0001F44A \U0001F3FE",
    ":oncoming_fist_medium-light_skin_tone:": "\U0001F44A \U0001F3FC",
    ":oncoming_fist_medium_skin_tone:": "\U0001F44A \U0001F3FD",
    ":oncoming_police_car:": "\U0001F694",
    ":oncoming_taxi:": "\U0001F696",
    ":one-thirty:": "\U0001F55C",
    ":one_o’clock:": "\U0001F550",
    ":open_book:": "\U0001F4D6",
    ":open_file_folder:": "\U0001F4C2",
    ":open_hands:": "\U0001F450",
    ":open_hands_dark_skin_tone:": "\U0001F450 \U0001F3FF",
    ":open_hands_light_skin_tone:": "\U0001F450 \U0001F3FB",
    ":open_hands_medium-dark_skin_tone:": "\U0001F450 \U0001F3FE",
    ":open_hands_medium-light_skin_tone:": "\U0001F450 \U0001F3FC",
    ":open_hands_medium_skin_tone:": "\U0001F450 \U0001F3FD",
    ":open_mailbox_with_lowered_flag:": "\U0001F4ED",
    ":open_mailbox_with_raised_flag:": "\U0001F4EC",
    ":optical_disk:": "\U0001F4BF",
    ":orange_book:": "\U0001F4D9",
    ":orthodox_cross:": "\U00002626",
    ":outbox_tray:": "\U0001F4E4",
    ":owl:": "\U0001F989",
    ":ox:": "\U0001F402",
    ":package:": "\U0001F4E6",
    ":page_facing_up:": "\U0001F4C4",
    ":page_with_curl:": "\U0001F4C3",
    ":pager:": "\U0001F4DF",
    ":paintbrush:": "\U0001F58C",
    ":palm_tree:": "\U0001F334",
    ":pancakes:": "\U0001F95E",
    ":panda_face:": "\U0001F43C",
    ":paperclip:": "\U0001F4CE",
    ":part_alternation_mark:": "\U0000303D",
    ":party_popper:": "\U0001F389",
    ":passenger_ship:": "\U0001F6F3",
    ":passport_control:": "\U0001F6C2",
    ":pause_button:": "\U000023F8",
    ":paw_prints:": "\U0001F43E",
    ":peace_symbol:": "\U0000262E",
    ":peach:": "\U0001F351",
    ":peanuts:": "\U0001F95C",
    ":pear:": "\U0001F350",
    ":pen:": "\U0001F58A",
    ":pencil:": "\U0000270F",
    ":penguin:": "\U0001F427",
    ":pensive_face:": "\U0001F614",
    ":people_with_bunny_ears_partying:": "\U0001F46F",
    ":people_wrestling:": "\U0001F93C",
    ":performing_arts:": "\U0001F3AD",
    ":persevering_face:": "\U0001F623",
    ":person_biking:": "\U0001F6B4",
    ":person_biking_dark_skin_tone:": "\U0001F6B4 \U0001F3FF",
    ":person_biking_light_skin_tone:": "\U0001F6B4 \U0001F3FB",
    ":person_biking_medium-dark_skin_tone:": "\U0001F6B4 \U0001F3FE",
    ":person_biking_medium-light_skin_tone:": "\U0001F6B4 \U0001F3FC",
    ":person_biking_medium_skin_tone:": "\U0001F6B4 \U0001F3FD",
    ":person_bouncing_ball:": "\U000026F9",
    ":person_bouncing_ball_dark_skin_tone:": "\U000026F9 \U0001F3FF",
    ":person_bouncing_ball_light_skin_tone:": "\U000026F9 \U0001F3FB",
    ":person_bouncing_ball_medium-dark_skin_tone:": "\U000026F9 \U0001F3FE",
    ":person_bouncing_ball_medium-light_skin_tone:": "\U000026F9 \U0001F3FC",
    ":person_bouncing_ball_medium_skin_tone:": "\U000026F9 \U0001F3FD",
    ":person_bowing:": "\U0001F647",
    ":person_bowing_dark_skin_tone:": "\U0001F647 \U0001F3FF",
    ":person_bowing_light_skin_tone:": "\U0001F647 \U0001F3FB",
    ":person_bowing_medium-dark_skin_tone:": "\U0001F647 \U0001F3FE",
    ":person_bowing_medium-light_skin_tone:": "\U0001F647 \U0001F3FC",
    ":person_bowing_medium_skin_tone:": "\U0001F647 \U0001F3FD",
    ":person_cartwheeling:": "\U0001F938",
    ":person_cartwheeling_dark_skin_tone:": "\U0001F938 \U0001F3FF",
    ":person_cartwheeling_light_skin_tone:": "\U0001F938 \U0001F3FB",
    ":person_cartwheeling_medium-dark_skin_tone:": "\U0001F938 \U0001F3FE",
    ":person_cartwheeling_medium-light_skin_tone:": "\U0001F938 \U0001F3FC",
    ":person_cartwheeling_medium_skin_tone:": "\U0001F938 \U0001F3FD",
    ":person_facepalming:": "\U0001F926",
    ":person_facepalming_dark_skin_tone:": "\U0001F926 \U0001F3FF",
    ":person_facepalming_light_skin_tone:": "\U0001F926 \U0001F3FB",
    ":person_facepalming_medium-dark_skin_tone:": "\U0001F926 \U0001F3FE",
    ":person_facepalming_medium-light_skin_tone:": "\U0001F926 \U0001F3FC",
    ":person_facepalming_medium_skin_tone:": "\U0001F926 \U0001F3FD",
    ":person_fencing:": "\U0001F93A",
    ":person_frowning:": "\U0001F64D",
    ":person_frowning_dark_skin_tone:": "\U0001F64D \U0001F3FF",
    ":person_frowning_light_skin_tone:": "\U0001F64D \U0001F3FB",
    ":person_frowning_medium-dark_skin_tone:": "\U0001F64D \U0001F3FE",
    ":person_frowning_medium-light_skin_tone:": "\U0001F64D \U0001F3FC",
    ":person_frowning_medium_skin_tone:": "\U0001F64D \U0001F3FD",
    ":person_gesturing_NO:": "\U0001F645",
    ":person_gesturing_NO_dark_skin_tone:": "\U0001F645 \U0001F3FF",
    ":person_gesturing_NO_light_skin_tone:": "\U0001F645 \U0001F3FB",
    ":person_gesturing_NO_medium-dark_skin_tone:": "\U0001F645 \U0001F3FE",
    ":person_gesturing_NO_medium-light_skin_tone:": "\U0001F645 \U0001F3FC",
    ":person_gesturing_NO_medium_skin_tone:": "\U0001F645 \U0001F3FD",
    ":person_gesturing_OK:": "\U0001F646",
    ":person_gesturing_OK_dark_skin_tone:": "\U0001F646 \U0001F3FF",
    ":person_gesturing_OK_light_skin_tone:": "\U0001F646 \U0001F3FB",
    ":person_gesturing_OK_medium-dark_skin_tone:": "\U0001F646 \U0001F3FE",
    ":person_gesturing_OK_medium-light_skin_tone:": "\U0001F646 \U0001F3FC",
    ":person_gesturing_OK_medium_skin_tone:": "\U0001F646 \U0001F3FD",
    ":person_getting_haircut:": "\U0001F487",
    ":person_getting_haircut_dark_skin_tone:": "\U0001F487 \U0001F3FF",
    ":person_getting_haircut_light_skin_tone:": "\U0001F487 \U0001F3FB",
    ":person_getting_haircut_medium-dark_skin_tone:": "\U0001F487 \U0001F3FE",
    ":person_getting_haircut_medium-light_skin_tone:": "\U0001F487 \U0001F3FC",
    ":person_getting_haircut_medium_skin_tone:": "\U0001F487 \U0001F3FD",
    ":person_getting_massage:": "\U0001F486",
    ":person_getting_massage_dark_skin_tone:": "\U0001F486 \U0001F3FF",
    ":person_getting_massage_light_skin_tone:": "\U0001F486 \U0001F3FB",
    ":person_getting_massage_medium-dark_skin_tone:": "\U0001F486 \U0001F3FE",
    ":person_getting_massage_medium-light_skin_tone:": "\U0001F486 \U0001F3FC",
    ":person_getting_massage_medium_skin_tone:": "\U0001F486 \U0001F3FD",
    ":person_golfing:": "\U0001F3CC",
    ":person_golfing_dark_skin_tone:": "\U0001F3CC \U0001F3FF",
    ":person_golfing_light_skin_tone:": "\U0001F3CC \U0001F3FB",
    ":person_golfing_medium-dark_skin_tone:": "\U0001F3CC \U0001F3FE",
    ":person_golfing_medium-light_skin_tone:": "\U0001F3CC \U0001F3FC",
    ":person_golfing_medium_skin_tone:": "\U0001F3CC \U0001F3FD",
    ":person_in_bed:": "\U0001F6CC",
    ":person_in_bed_dark_skin_tone:": "\U0001F6CC \U0001F3FF",
    ":person_in_bed_light_skin_tone:": "\U0001F6CC \U0001F3FB",
    ":person_in_bed_medium-dark_skin_tone:": "\U0001F6CC \U0001F3FE",
    ":person_in_bed_medium-light_skin_tone:": "\U0001F6CC \U0001F3FC",
    ":person_in_bed_medium_skin_tone:": "\U0001F6CC \U0001F3FD",
    ":person_juggling:": "\U0001F939",
    ":person_juggling_dark_skin_tone:": "\U0001F939 \U0001F3FF",
    ":person_juggling_light_skin_tone:": "\U0001F939 \U0001F3FB",
    ":person_juggling_medium-dark_skin_tone:": "\U0001F939 \U0001F3FE",
    ":person_juggling_medium-light_skin_tone:": "\U0001F939 \U0001F3FC",
    ":person_juggling_medium_skin_tone:": "\U0001F939 \U0001F3FD",
    ":person_lifting_weights:": "\U0001F3CB",
    ":person_lifting_weights_dark_skin_tone:": "\U0001F3CB \U0001F3FF",
    ":person_lifting_weights_light_skin_tone:": "\U0001F3CB \U0001F3FB",
    ":person_lifting_weights_medium-dark_skin_tone:": "\U0001F3CB \U0001F3FE",
    ":person_lifting_weights_medium-light_skin_tone:": "\U0001F3CB \U0001F3FC",
    ":person_lifting_weights_medium_skin_tone:": "\U0001F3CB \U0001F3FD",
    ":person_mountain_biking:": "\U0001F6B5",
    ":person_mountain_biking_dark_skin_tone:": "\U0001F6B5 \U0001F3FF",
    ":person_mountain_biking_light_skin_tone:": "\U0001F6B5 \U0001F3FB",
    ":person_mountain_biking_medium-dark_skin_tone:": "\U0001F6B5 \U0001F3FE",
    ":person_mountain_biking_medium-light_skin_tone:": "\U0001F6B5 \U0001F3FC",
    ":person_mountain_biking_medium_skin_tone:": "\U0001F6B5 \U0001F3FD",
    ":person_playing_handball:": "\U0001F93E",
    ":person_playing_handball_dark_skin_tone:": "\U0001F93E \U0001F3FF",
    ":person_playing_handball_light_skin_tone:": "\U0001F93E \U0001F3FB",
    ":person_playing_handball_medium-dark_skin_tone:": "\U0001F93E \U0001F3FE",
    ":person_playing_handball_medium-light_skin_tone:": "\U0001F93E \U0001F3FC",
    ":person_playing_handball_medium_skin_tone:": "\U0001F93E \U0001F3FD",
    ":person_playing_water_polo:": "\U0001F93D",
    ":person_playing_water_polo_dark_skin_tone:": "\U0001F93D \U0001F3FF",
    ":person_playing_water_polo_light_skin_tone:": "\U0001F93D \U0001F3FB",
    ":person_playing_water_polo_medium-dark_skin_tone:": "\U0001F93D \U0001F3FE",
    ":person_playing_water_polo_medium-light_skin_tone:": "\U0001F93D \U0001F3FC",
    ":person_playing_water_polo_medium_skin_tone:": "\U0001F93D \U0001F3FD",
    ":person_pouting:": "\U0001F64E",
    ":person_pouting_dark_skin_tone:": "\U0001F64E \U0001F3FF",
    ":person_pouting_light_skin_tone:": "\U0001F64E \U0001F3FB",
    ":person_pouting_medium-dark_skin_tone:": "\U0001F64E \U0001F3FE",
    ":person_pouting_medium-light_skin_tone:": "\U0001F64E \U0001F3FC",
    ":person_pouting_medium_skin_tone:": "\U0001F64E \U0001F3FD",
    ":person_raising_hand:": "\U0001F64B",
    ":person_raising_hand_dark_skin_tone:": "\U0001F64B \U0001F3FF",
    ":person_raising_hand_light_skin_tone:": "\U0001F64B \U0001F3FB",
    ":person_raising_hand_medium-dark_skin_tone:": "\U0001F64B \U0001F3FE",
    ":person_raising_hand_medium-light_skin_tone:": "\U0001F64B \U0001F3FC",
    ":person_raising_hand_medium_skin_tone:": "\U0001F64B \U0001F3FD",
    ":person_rowing_boat:": "\U0001F6A3",
    ":person_rowing_boat_dark_skin_tone:": "\U0001F6A3 \U0001F3FF",
    ":person_rowing_boat_light_skin_tone:": "\U0001F6A3 \U0001F3FB",
    ":person_rowing_boat_medium-dark_skin_tone:": "\U0001F6A3 \U0001F3FE",
    ":person_rowing_boat_medium-light_skin_tone:": "\U0001F6A3 \U0001F3FC",
    ":person_rowing_boat_medium_skin_tone:": "\U0001F6A3 \U0001F3FD",
    ":person_running:": "\U0001F3C3",
    ":person_running_dark_skin_tone:": "\U0001F3C3 \U0001F3FF",
    ":person_running_light_skin_tone:": "\U0001F3C3 \U0001F3FB",
    ":person_running_medium-dark_skin_tone:": "\U0001F3C3 \U0001F3FE",
    ":person_running_medium-light_skin_tone:": "\U0001F3C3 \U0001F3FC",
    ":person_running_medium_skin_tone:": "\U0001F3C3 \U0001F3FD",
    ":person_shrugging:": "\U0001F937",
    ":person_shrugging_dark_skin_tone:": "\U0001F937 \U0001F3FF",
    ":person_shrugging_light_skin_tone:": "\U0001F937 \U0001F3FB",
    ":person_shrugging_medium-dark_skin_tone:": "\U0001F937 \U0001F3FE",
    ":person_shrugging_medium-light_skin_tone:": "\U0001F937 \U0001F3FC",
    ":person_shrugging_medium_skin_tone:": "\U0001F937 \U0001F3FD",
    ":person_surfing:": "\U0001F3C4",
    ":person_surfing_dark_skin_tone:": "\U0001F3C4 \U0001F3FF",
    ":person_surfing_light_skin_tone:": "\U0001F3C4 \U0001F3FB",
    ":person_surfing_medium-dark_skin_tone:": "\U0001F3C4 \U0001F3FE",
    ":person_surfing_medium-light_skin_tone:": "\U0001F3C4 \U0001F3FC",
    ":person_surfing_medium_skin_tone:": "\U0001F3C4 \U0001F3FD",
    ":person_swimming:": "\U0001F3CA",
    ":person_swimming_dark_skin_tone:": "\U0001F3CA \U0001F3FF",
    ":person_swimming_light_skin_tone:": "\U0001F3CA \U0001F3FB",
    ":person_swimming_medium-dark_skin_tone:": "\U0001F3CA \U0001F3FE",
    ":person_swimming_medium-light_skin_tone:": "\U0001F3CA \U0001F3FC",
    ":person_swimming_medium_skin_tone:": "\U0001F3CA \U0001F3FD",
    ":person_taking_bath:": "\U0001F6C0",
    ":person_taking_bath_dark_skin_tone:": "\U0001F6C0 \U0001F3FF",
    ":person_taking_bath_light_skin_tone:": "\U0001F6C0 \U0001F3FB",
    ":person_taking_bath_medium-dark_skin_tone:": "\U0001F6C0 \U0001F3FE",
    ":person_taking_bath_medium-light_skin_tone:": "\U0001F6C0 \U0001F3FC",
    ":person_taking_bath_medium_skin_tone:": "\U0001F6C0 \U0001F3FD",
    ":person_tipping_hand:": "\U0001F481",
    ":person_tipping_hand_dark_skin_tone:": "\U0001F481 \U0001F3FF",
    ":person_tipping_hand_light_skin_tone:": "\U0001F481 \U0001F3FB",
    ":person_tipping_hand_medium-dark_skin_tone:": "\U0001F481 \U0001F3FE",
    ":person_tipping_hand_medium-light_skin_tone:": "\U0001F481 \U0001F3FC",
    ":person_tipping_hand_medium_skin_tone:": "\U0001F481 \U0001F3FD",
    ":person_walking:": "\U0001F6B6",
    ":person_walking_dark_skin_tone:": "\U0001F6B6 \U0001F3FF",
    ":person_walking_light_skin_tone:": "\U0001F6B6 \U0001F3FB",
    ":person_walking_medium-dark_skin_tone:": "\U0001F6B6 \U0001F3FE",
    ":person_walking_medium-light_skin_tone:": "\U0001F6B6 \U0001F3FC",
    ":person_walking_medium_skin_tone:": "\U0001F6B6 \U0001F3FD",
    ":person_wearing_turban:": "\U0001F473",
    ":person_wearing_turban_dark_skin_tone:": "\U0001F473 \U0001F3FF",
    ":person_wearing_turban_light_skin_tone:": "\U0001F473 \U0001F3FB",
    ":person_wearing_turban_medium-dark_skin_tone:": "\U0001F473 \U0001F3FE",
    ":person_wearing_turban_medium-light_skin_tone:": "\U0001F473 \U0001F3FC",
    ":person_wearing_turban_medium_skin_tone:": "\U0001F473 \U0001F3FD",
    ":pick:": "\U000026CF",
    ":pig:": "\U0001F416",
    ":pig_face:": "\U0001F437",
    ":pig_nose:": "\U0001F43D",
    ":pile_of_poo:": "\U0001F4A9",
    ":pill:": "\U0001F48A",
    ":pine_decoration:": "\U0001F38D",
    ":pineapple:": "\U0001F34D",
    ":ping_pong:": "\U0001F3D3",
    ":pistol:": "\U0001F52B",
    ":pizza:": "\U0001F355",
    ":place_of_worship:": "\U0001F6D0",
    ":play_button:": "\U000025B6",
    ":play_or_pause_button:": "\U000023EF",
    ":police_car:": "\U0001F693",
    ":police_car_light:": "\U0001F6A8",
    ":police_officer:": "\U0001F46E",
    ":police_officer_dark_skin_tone:": "\U0001F46E \U0001F3FF",
    ":police_officer_light_skin_tone:": "\U0001F46E \U0001F3FB",
    ":police_officer_medium-dark_skin_tone:": "\U0001F46E \U0001F3FE",
    ":police_officer_medium-light_skin_tone:": "\U0001F46E \U0001F3FC",
    ":police_officer_medium_skin_tone:": "\U0001F46E \U0001F3FD",
    ":poodle:": "\U0001F429",
    ":pool_8_ball:": "\U0001F3B1",
    ":popcorn:": "\U0001F37F",
    ":post_office:": "\U0001F3E4",
    ":postal_horn:": "\U0001F4EF",
    ":postbox:": "\U0001F4EE",
    ":pot_of_food:": "\U0001F372",
    ":potable_water:": "\U0001F6B0",
    ":potato:": "\U0001F954",
    ":poultry_leg:": "\U0001F357",
    ":pound_banknote:": "\U0001F4B7",
    ":pouting_cat_face:": "\U0001F63E",
    ":pouting_face:": "\U0001F621",
    ":prayer_beads:": "\U0001F4FF",
    ":pregnant_woman:": "\U0001F930",
    ":pregnant_woman_dark_skin_tone:": "\U0001F930 \U0001F3FF",
    ":pregnant_woman_light_skin_tone:": "\U0001F930 \U0001F3FB",
    ":pregnant_woman_medium-dark_skin_tone:": "\U0001F930 \U0001F3FE",
    ":pregnant_woman_medium-light_skin_tone:": "\U0001F930 \U0001F3FC",
    ":pregnant_woman_medium_skin_tone:": "\U0001F930 \U0001F3FD",
    ":prince:": "\U0001F934",
    ":prince_dark_skin_tone:": "\U0001F934 \U0001F3FF",
    ":prince_light_skin_tone:": "\U0001F934 \U0001F3FB",
    ":prince_medium-dark_skin_tone:": "\U0001F934 \U0001F3FE",
    ":prince_medium-light_skin_tone:": "\U0001F934 \U0001F3FC",
    ":prince_medium_skin_tone:": "\U0001F934 \U0001F3FD",
    ":princess:": "\U0001F478",
    ":princess_dark_skin_tone:": "\U0001F478 \U0001F3FF",
    ":princess_light_skin_tone:": "\U0001F478 \U0001F3FB",
    ":princess_medium-dark_skin_tone:": "\U0001F478 \U0001F3FE",
    ":princess_medium-light_skin_tone:": "\U0001F478 \U0001F3FC",
    ":princess_medium_skin_tone:": "\U0001F478 \U0001F3FD",
    ":printer:": "\U0001F5A8",
    ":prohibited:": "\U0001F6AB",
    ":purple_heart:": "\U0001F49C",
    ":purse:": "\U0001F45B",
    ":pushpin:": "\U0001F4CC",
    ":question_mark:": "\U00002753",
    ":rabbit:": "\U0001F407",
    ":rabbit_face:": "\U0001F430",
    ":racing_car:": "\U0001F3CE",
    ":radio:": "\U0001F4FB",
    ":radio_button:": "\U0001F518",
    ":radioactive:": "\U00002622",
    ":railway_car:": "\U0001F683",
    ":railway_track:": "\U0001F6E4",
    ":rainbow:": "\U0001F308",
    ":rainbow_flag:": "\U0001F3F3 \U0000FE0F \U0000200D \U0001F308",
    ":raised_back_of_hand:": "\U0001F91A",
    ":raised_back_of_hand_dark_skin_tone:": "\U0001F91A \U0001F3FF",
    ":raised_back_of_hand_light_skin_tone:": "\U0001F91A \U0001F3FB",
    ":raised_back_of_hand_medium-dark_skin_tone:": "\U0001F91A \U0001F3FE",
    ":raised_back_of_hand_medium-light_skin_tone:": "\U0001F91A \U0001F3FC",
    ":raised_back_of_hand_medium_skin_tone:": "\U0001F91A \U0001F3FD",
    ":raised_fist:": "\U0000270A",
    ":raised_fist_dark_skin_tone:": "\U0000270A \U0001F3FF",
    ":raised_fist_light_skin_tone:": "\U0000270A \U0001F3FB",
    ":raised_fist_medium-dark_skin_tone:": "\U0000270A \U0001F3FE",
    ":raised_fist_medium-light_skin_tone:": "\U0000270A \U0001F3FC",
    ":raised_fist_medium_skin_tone:": "\U0000270A \U0001F3FD",
    ":raised_hand:": "\U0000270B",
    ":raised_hand_dark_skin_tone:": "\U0000270B \U0001F3FF",
    ":raised_hand_light_skin_tone:": "\U0000270B \U0001F3FB",
    ":raised_hand_medium-dark_skin_tone:": "\U0000270B \U0001F3FE",
    ":raised_hand_medium-light_skin_tone:": "\U0000270B \U0001F3FC",
    ":raised_hand_medium_skin_tone:": "\U0000270B \U0001F3FD",
    ":raised_hand_with_fingers_splayed:": "\U0001F590",
    ":raised_hand_with_fingers_splayed_dark_skin_tone:": "\U0001F590 \U0001F3FF",
    ":raised_hand_with_fingers_splayed_light_skin_tone:": "\U0001F590 \U0001F3FB",
    ":raised_hand_with_fingers_splayed_medium-dark_skin_tone:": "\U0001F590 \U0001F3FE",
    ":raised_hand_with_fingers_splayed_medium-light_skin_tone:": "\U0001F590 \U0001F3FC",
    ":raised_hand_with_fingers_splayed_medium_skin_tone:": "\U0001F590 \U0001F3FD",
    ":raising_hands:": "\U0001F64C",
    ":raising_hands_dark_skin_tone:": "\U0001F64C \U0001F3FF",
    ":raising_hands_light_skin_tone:": "\U0001F64C \U0001F3FB",
    ":raising_hands_medium-dark_skin_tone:": "\U0001F64C \U0001F3FE",
    ":raising_hands_medium-light_skin_tone:": "\U0001F64C \U0001F3FC",
    ":raising_hands_medium_skin_tone:": "\U0001F64C \U0001F3FD",
    ":ram:": "\U0001F40F",
    ":rat:": "\U0001F400",
    ":record_button:": "\U000023FA",
    ":recycling_symbol:": "\U0000267B",
    ":red_apple:": "\U0001F34E",
    ":red_circle:": "\U0001F534",
    ":red_heart:": "\U00002764",
    ":red_paper_lantern:": "\U0001F3EE",
    ":red_triangle_pointed_down:": "\U0001F53B",
    ":red_triangle_pointed_up:": "\U0001F53A",
    ":registered:": "\U000000AE",
    ":relieved_face:": "\U0001F60C",
    ":reminder_ribbon:": "\U0001F397",
    ":repeat_button:": "\U0001F501",
    ":repeat_single_button:": "\U0001F502",
    ":rescue_worker’s_helmet:": "\U000026D1",
    ":restroom:": "\U0001F6BB",
    ":reverse_button:": "\U000025C0",
    ":revolving_hearts:": "\U0001F49E",
    ":rhinoceros:": "\U0001F98F",
    ":ribbon:": "\U0001F380",
    ":rice_ball:": "\U0001F359",
    ":rice_cracker:": "\U0001F358",
    ":right-facing_fist:": "\U0001F91C",
    ":right-facing_fist_dark_skin_tone:": "\U0001F91C \U0001F3FF",
    ":right-facing_fist_light_skin_tone:": "\U0001F91C \U0001F3FB",
    ":right-facing_fist_medium-dark_skin_tone:": "\U0001F91C \U0001F3FE",
    ":right-facing_fist_medium-light_skin_tone:": "\U0001F91C \U0001F3FC",
    ":right-facing_fist_medium_skin_tone:": "\U0001F91C \U0001F3FD",
    ":right-pointing_magnifying_glass:": "\U0001F50E",
    ":right_anger_bubble:": "\U0001F5EF",
    ":right_arrow:": "\U000027A1",
    ":right_arrow_curving_down:": "\U00002935",
    ":right_arrow_curving_left:": "\U000021A9",
    ":right_arrow_curving_up:": "\U00002934",
    ":ring:": "\U0001F48D",
    ":roasted_sweet_potato:": "\U0001F360",
    ":robot_face:": "\U0001F916",
    ":rocket:": "\U0001F680",
    ":rolled-up_newspaper:": "\U0001F5DE",
    ":roller_coaster:": "\U0001F3A2",
    ":rolling_on_the_floor_laughing:": "\U0001F923",
    ":rooster:": "\U0001F413",
    ":rose:": "\U0001F339",
    ":rosette:": "\U0001F3F5",
    ":round_pushpin:": "\U0001F4CD",
    ":rugby_football:": "\U0001F3C9",
    ":running_shirt:": "\U0001F3BD",
    ":running_shoe:": "\U0001F45F",
    ":sailboat:": "\U000026F5",
    ":sake:": "\U0001F376",
    ":satellite:": "\U0001F6F0",
    ":satellite_antenna:": "\U0001F4E1",
    ":saxophone:": "\U0001F3B7",
    ":school:": "\U0001F3EB",
    ":school_backpack:": "\U0001F392",
    ":scissors:": "\U00002702",
    ":scorpion:": "\U0001F982",
    ":scroll:": "\U0001F4DC",
    ":seat:": "\U0001F4BA",
    ":see-no-evil_monkey:": "\U0001F648",
    ":seedling:": "\U0001F331",
    ":selfie:": "\U0001F933",
    ":selfie_dark_skin_tone:": "\U0001F933 \U0001F3FF",
    ":selfie_light_skin_tone:": "\U0001F933 \U0001F3FB",
    ":selfie_medium-dark_skin_tone:": "\U0001F933 \U0001F3FE",
    ":selfie_medium-light_skin_tone:": "\U0001F933 \U0001F3FC",
    ":selfie_medium_skin_tone:": "\U0001F933 \U0001F3FD",
    ":seven-thirty:": "\U0001F562",
    ":seven_o’clock:": "\U0001F556",
    ":shallow_pan_of_food:": "\U0001F958",
    ":shamrock:": "\U00002618",
    ":shark:": "\U0001F988",
    ":shaved_ice:": "\U0001F367",
    ":sheaf_of_rice:": "\U0001F33E",
    ":sheep:": "\U0001F411",
    ":shield:": "\U0001F6E1",
    ":shinto_shrine:": "\U000026E9",
    ":ship:": "\U0001F6A2",
    ":shooting_star:": "\U0001F320",
    ":shopping_bags:": "\U0001F6CD",
    ":shopping_cart:": "\U0001F6D2",
    ":shortcake:": "\U0001F370",
    ":shower:": "\U0001F6BF",
    ":shrimp:": "\U0001F990",
    ":shuffle_tracks_button:": "\U0001F500",
    ":sign_of_the_horns:": "\U0001F918",
    ":sign_of_the_horns_dark_skin_tone:": "\U0001F918 \U0001F3FF",
    ":sign_of_the_horns_light_skin_tone:": "\U0001F918 \U0001F3FB",
    ":sign_of_the_horns_medium-dark_skin_tone:": "\U0001F918 \U0001F3FE",
    ":sign_of_the_horns_medium-light_skin_tone:": "\U0001F918 \U0001F3FC",
    ":sign_of_the_horns_medium_skin_tone:": "\U0001F918 \U0001F3FD",
    ":six-thirty:": "\U0001F561",
    ":six_o’clock:": "\U0001F555",
    ":skier:": "\U000026F7",
    ":skis:": "\U0001F3BF",
    ":skull:": "\U0001F480",
    ":skull_and_crossbones:": "\U00002620",
    ":sleeping_face:": "\U0001F634",
    ":sleepy_face:": "\U0001F62A",
    ":slightly_frowning_face:": "\U0001F641",
    ":slightly_smiling_face:": "\U0001F642",
    ":slot_machine:": "\U0001F3B0",
    ":small_airplane:": "\U0001F6E9",
    ":small_blue_diamond:": "\U0001F539",
    ":small_orange_diamond:": "\U0001F538",
    ":smiling_cat_face_with_heart-eyes:": "\U0001F63B",
    ":smiling_cat_face_with_open_mouth:": "\U0001F63A",
    ":smiling_face:": "\U0000263A",
    ":smiling_face_with_halo:": "\U0001F607",
    ":smiling_face_with_heart-eyes:": "\U0001F60D",
    ":smiling_face_with_horns:": "\U0001F608",
    ":smiling_face_with_open_mouth:": "\U0001F603",
    ":smiling_face_with_open_mouth_&_closed_eyes:": "\U0001F606",
    ":smiling_face_with_open_mouth_&_cold_sweat:": "\U0001F605",
    ":smiling_face_with_open_mouth_&_smiling_eyes:": "\U0001F604",
    ":smiling_face_with_smiling_eyes:": "\U0001F60A",
    ":smiling_face_with_sunglasses:": "\U0001F60E",
    ":smirking_face:": "\U0001F60F",
    ":snail:": "\U0001F40C",
    ":snake:": "\U0001F40D",
    ":sneezing_face:": "\U0001F927",
    ":snow-capped_mountain:": "\U0001F3D4",
    ":snowboarder:": "\U0001F3C2",
    ":snowboarder_dark_skin_tone:": "\U0001F3C2 \U0001F3FF",
    ":snowboarder_light_skin_tone:": "\U0001F3C2 \U0001F3FB",
    ":snowboarder_medium-dark_skin_tone:": "\U0001F3C2 \U0001F3FE",
    ":snowboarder_medium-light_skin_tone:": "\U0001F3C2 \U0001F3FC",
    ":snowboarder_medium_skin_tone:": "\U0001F3C2 \U0001F3FD",
    ":snowflake:": "\U00002744",
    ":snowman:": "\U00002603",
    ":snowman_without_snow:": "\U000026C4",
    ":soccer_ball:": "\U000026BD",
    ":soft_ice_cream:": "\U0001F366",
    ":spade_suit:": "\U00002660",
    ":spaghetti:": "\U0001F35D",
    ":sparkle:": "\U00002747",
    ":sparkler:": "\U0001F387",
    ":sparkles:": "\U00002728",
    ":sparkling_heart:": "\U0001F496",
    ":speak-no-evil_monkey:": "\U0001F64A",
    ":speaker_high_volume:": "\U0001F50A",
    ":speaker_low_volume:": "\U0001F508",
    ":speaker_medium_volume:": "\U0001F509",
    ":speaking_head:": "\U0001F5E3",
    ":speech_balloon:": "\U0001F4AC",
    ":speedboat:": "\U0001F6A4",
    ":spider:": "\U0001F577",
    ":spider_web:": "\U0001F578",
    ":spiral_calendar:": "\U0001F5D3",
    ":spiral_notepad:": "\U0001F5D2",
    ":spiral_shell:": "\U0001F41A",
    ":spoon:": "\U0001F944",
    ":sport_utility_vehicle:": "\U0001F699",
    ":sports_medal:": "\U0001F3C5",
    ":spouting_whale:": "\U0001F433",
    ":squid:": "\U0001F991",
    ":stadium:": "\U0001F3DF",
    ":star_and_crescent:": "\U0000262A",
    ":star_of_David:": "\U00002721",
    ":station:": "\U0001F689",
    ":steaming_bowl:": "\U0001F35C",
    ":stop_button:": "\U000023F9",
    ":stop_sign:": "\U0001F6D1",
    ":stopwatch:": "\U000023F1",
    ":straight_ruler:": "\U0001F4CF",
    ":strawberry:": "\U0001F353",
    ":studio_microphone:": "\U0001F399",
    ":stuffed_flatbread:": "\U0001F959",
    ":sun:": "\U00002600",
    ":sun_behind_cloud:": "\U000026C5",
    ":sun_behind_large_cloud:": "\U0001F325",
    ":sun_behind_rain_cloud:": "\U0001F326",
    ":sun_behind_small_cloud:": "\U0001F324",
    ":sun_with_face:": "\U0001F31E",
    ":sunflower:": "\U0001F33B",
    ":sunglasses:": "\U0001F576",
    ":sunrise:": "\U0001F305",
    ":sunrise_over_mountains:": "\U0001F304",
    ":sunset:": "\U0001F307",
    ":sushi:": "\U0001F363",
    ":suspension_railway:": "\U0001F69F",
    ":sweat_droplets:": "\U0001F4A6",
    ":synagogue:": "\U0001F54D",
    ":syringe:": "\U0001F489",
    ":t-shirt:": "\U0001F455",
    ":taco:": "\U0001F32E",
    ":tanabata_tree:": "\U0001F38B",
    ":tangerine:": "\U0001F34A",
    ":taxi:": "\U0001F695",
    ":teacup_without_handle:": "\U0001F375",
    ":tear-off_calendar:": "\U0001F4C6",
    ":telephone:": "\U0000260E",
    ":telephone_receiver:": "\U0001F4DE",
    ":telescope:": "\U0001F52D",
    ":television:": "\U0001F4FA",
    ":ten-thirty:": "\U0001F565",
    ":ten_o’clock:": "\U0001F559",
    ":tennis:": "\U0001F3BE",
    ":tent:": "\U000026FA",
    ":thermometer:": "\U0001F321",
    ":thinking_face:": "\U0001F914",
    ":thought_balloon:": "\U0001F4AD",
    ":three-thirty:": "\U0001F55E",
    ":three_o’clock:": "\U0001F552",
    ":thumbs_down:": "\U0001F44E",
    ":thumbs_down_dark_skin_tone:": "\U0001F44E \U0001F3FF",
    ":thumbs_down_light_skin_tone:": "\U0001F44E \U0001F3FB",
    ":thumbs_down_medium-dark_skin_tone:": "\U0001F44E \U0001F3FE",
    ":thumbs_down_medium-light_skin_tone:": "\U0001F44E \U0001F3FC",
    ":thumbs_down_medium_skin_tone:": "\U0001F44E \U0001F3FD",
    ":thumbs_up:": "\U0001F44D",
    ":thumbs_up_dark_skin_tone:": "\U0001F44D \U0001F3FF",
    ":thumbs_up_light_skin_tone:": "\U0001F44D \U0001F3FB",
    ":thumbs_up_medium-dark_skin_tone:": "\U0001F44D \U0001F3FE",
    ":thumbs_up_medium-light_skin_tone:": "\U0001F44D \U0001F3FC",
    ":thumbs_up_medium_skin_tone:": "\U0001F44D \U0001F3FD",
    ":ticket:": "\U0001F3AB",
    ":tiger:": "\U0001F405",
    ":tiger_face:": "\U0001F42F",
    ":timer_clock:": "\U000023F2",
    ":tired_face:": "\U0001F62B",
    ":toilet:": "\U0001F6BD",
    ":tomato:": "\U0001F345",
    ":tongue:": "\U0001F445",
    ":top_hat:": "\U0001F3A9",
    ":tornado:": "\U0001F32A",
    ":trackball:": "\U0001F5B2",
    ":tractor:": "\U0001F69C",
    ":trade_mark:": "\U00002122",
    ":train:": "\U0001F686",
    ":tram:": "\U0001F68A",
    ":tram_car:": "\U0001F68B",
    ":triangular_flag:": "\U0001F6A9",
    ":triangular_ruler:": "\U0001F4D0",
    ":trident_emblem:": "\U0001F531",
    ":trolleybus:": "\U0001F68E",
    ":trophy:": "\U0001F3C6",
    ":tropical_drink:": "\U0001F379",
    ":tropical_fish:": "\U0001F420",
    ":trumpet:": "\U0001F3BA",
    ":tulip:": "\U0001F337",
    ":tumbler_glass:": "\U0001F943",
    ":turkey:": "\U0001F983",
    ":turtle:": "\U0001F422",
    ":twelve-thirty:": "\U0001F567",
    ":twelve_o’clock:": "\U0001F55B",
    ":two-hump_camel:": "\U0001F42B",
    ":two-thirty:": "\U0001F55D",
    ":two_hearts:": "\U0001F495",
    ":two_men_holding_hands:": "\U0001F46C",
    ":two_o’clock:": "\U0001F551",
    ":two_women_holding_hands:": "\U0001F46D",
    ":umbrella:": "\U00002602",
    ":umbrella_on_ground:": "\U000026F1",
    ":umbrella_with_rain_drops:": "\U00002614",
    ":unamused_face:": "\U0001F612",
    ":unicorn_face:": "\U0001F984",
    ":unlocked:": "\U0001F513",
    ":up-down_arrow:": "\U00002195",
    ":up-left_arrow:": "\U00002196",
    ":up-right_arrow:": "\U00002197",
    ":up_arrow:": "\U00002B06",
    ":up_button:": "\U0001F53C",
    ":upside-down_face:": "\U0001F643",
    ":vertical_traffic_light:": "\U0001F6A6",
    ":vibration_mode:": "\U0001F4F3",
    ":victory_hand:": "\U0000270C",
    ":victory_hand_dark_skin_tone:": "\U0000270C \U0001F3FF",
    ":victory_hand_light_skin_tone:": "\U0000270C \U0001F3FB",
    ":victory_hand_medium-dark_skin_tone:": "\U0000270C \U0001F3FE",
    ":victory_hand_medium-light_skin_tone:": "\U0000270C \U0001F3FC",
    ":victory_hand_medium_skin_tone:": "\U0000270C \U0001F3FD",
    ":video_camera:": "\U0001F4F9",
    ":video_game:": "\U0001F3AE",
    ":videocassette:": "\U0001F4FC",
    ":violin:": "\U0001F3BB",
    ":volcano:": "\U0001F30B",
    ":volleyball:": "\U0001F3D0",
    ":vulcan_salute:": "\U0001F596",
    ":vulcan_salute_dark_skin_tone:": "\U0001F596 \U0001F3FF",
    ":vulcan_salute_light_skin_tone:": "\U0001F596 \U0001F3FB",
    ":vulcan_salute_medium-dark_skin_tone:": "\U0001F596 \U0001F3FE",
    ":vulcan_salute_medium-light_skin_tone:": "\U0001F596 \U0001F3FC",
    ":vulcan_salute_medium_skin_tone:": "\U0001F596 \U0001F3FD",
    ":waning_crescent_moon:": "\U0001F318",
    ":waning_gibbous_moon:": "\U0001F316",
    ":warning:": "\U000026A0",
    ":wastebasket:": "\U0001F5D1",
    ":watch:": "\U0000231A",
    ":water_buffalo:": "\U0001F403",
    ":water_closet:": "\U0001F6BE",
    ":water_wave:": "\U0001F30A",
    ":watermelon:": "\U0001F349",
    ":waving_hand:": "\U0001F44B",
    ":waving_hand_dark_skin_tone:": "\U0001F44B \U0001F3FF",
    ":waving_hand_light_skin_tone:": "\U0001F44B \U0001F3FB",
    ":waving_hand_medium-dark_skin_tone:": "\U0001F44B \U0001F3FE",
    ":waving_hand_medium-light_skin_tone:": "\U0001F44B \U0001F3FC",
    ":waving_hand_medium_skin_tone:": "\U0001F44B \U0001F3FD",
    ":wavy_dash:": "\U00003030",
    ":waxing_crescent_moon:": "\U0001F312",
    ":waxing_gibbous_moon:": "\U0001F314",
    ":weary_cat_face:": "\U0001F640",
    ":weary_face:": "\U0001F629",
    ":wedding:": "\U0001F492",
    ":whale:": "\U0001F40B",
    ":wheel_of_dharma:": "\U00002638",
    ":wheelchair_symbol:": "\U0000267F",
    ":white_circle:": "\U000026AA",
    ":white_exclamation_mark:": "\U00002755",
    ":white_flag:": "\U0001F3F3",
    ":white_flower:": "\U0001F4AE",
    ":white_heavy_check_mark:": "\U00002705",
    ":white_large_square:": "\U00002B1C",
    ":white_medium-small_square:": "\U000025FD",
    ":white_medium_square:": "\U000025FB",
    ":white_medium_star:": "\U00002B50",
    ":white_question_mark:": "\U00002754",
    ":white_small_square:": "\U000025AB",
    ":white_square_button:": "\U0001F533",
    ":wilted_flower:": "\U0001F940",
    ":wind_chime:": "\U0001F390",
    ":wind_face:": "\U0001F32C",
    ":wine_glass:": "\U0001F377",
    ":winking_face:": "\U0001F609",
    ":wolf_face:": "\U0001F43A",
    ":woman:": "\U0001F469",
    ":woman_artist:": "\U0001F469 \U0000200D \U0001F3A8",
    ":woman_artist_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F3A8",
    ":woman_artist_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F3A8",
    ":woman_artist_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F3A8",
    ":woman_artist_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F3A8",
    ":woman_artist_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F3A8",
    ":woman_astronaut:": "\U0001F469 \U0000200D \U0001F680",
    ":woman_astronaut_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F680",
    ":woman_astronaut_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F680",
    ":woman_astronaut_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F680",
    ":woman_astronaut_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F680",
    ":woman_astronaut_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F680",
    ":woman_biking:": "\U0001F6B4 \U0000200D \U00002640 \U0000FE0F",
    ":woman_biking_dark_skin_tone:": "\U0001F6B4 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_biking_light_skin_tone:": "\U0001F6B4 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_biking_medium-dark_skin_tone:": "\U0001F6B4 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_biking_medium-light_skin_tone:": "\U0001F6B4 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_biking_medium_skin_tone:": "\U0001F6B4 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_bouncing_ball:": "\U000026F9 \U0000FE0F \U0000200D \U00002640 \U0000FE0F",
    ":woman_bouncing_ball_dark_skin_tone:": "\U000026F9 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_bouncing_ball_light_skin_tone:": "\U000026F9 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_bouncing_ball_medium-dark_skin_tone:": "\U000026F9 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_bouncing_ball_medium-light_skin_tone:": "\U000026F9 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_bouncing_ball_medium_skin_tone:": "\U000026F9 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_bowing:": "\U0001F647 \U0000200D \U00002640 \U0000FE0F",
    ":woman_bowing_dark_skin_tone:": "\U0001F647 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_bowing_light_skin_tone:": "\U0001F647 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_bowing_medium-dark_skin_tone:": "\U0001F647 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_bowing_medium-light_skin_tone:": "\U0001F647 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_bowing_medium_skin_tone:": "\U0001F647 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_cartwheeling:": "\U0001F938 \U0000200D \U00002640 \U0000FE0F",
    ":woman_cartwheeling_dark_skin_tone:": "\U0001F938 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_cartwheeling_light_skin_tone:": "\U0001F938 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_cartwheeling_medium-dark_skin_tone:": "\U0001F938 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_cartwheeling_medium-light_skin_tone:": "\U0001F938 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_cartwheeling_medium_skin_tone:": "\U0001F938 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_construction_worker:": "\U0001F477 \U0000200D \U00002640 \U0000FE0F",
    ":woman_construction_worker_dark_skin_tone:": "\U0001F477 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_construction_worker_light_skin_tone:": "\U0001F477 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_construction_worker_medium-dark_skin_tone:": "\U0001F477 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_construction_worker_medium-light_skin_tone:": "\U0001F477 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_construction_worker_medium_skin_tone:": "\U0001F477 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_cook:": "\U0001F469 \U0000200D \U0001F373",
    ":woman_cook_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F373",
    ":woman_cook_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F373",
    ":woman_cook_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F373",
    ":woman_cook_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F373",
    ":woman_cook_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F373",
    ":woman_dancing:": "\U0001F483",
    ":woman_dancing_dark_skin_tone:": "\U0001F483 \U0001F3FF",
    ":woman_dancing_light_skin_tone:": "\U0001F483 \U0001F3FB",
    ":woman_dancing_medium-dark_skin_tone:": "\U0001F483 \U0001F3FE",
    ":woman_dancing_medium-light_skin_tone:": "\U0001F483 \U0001F3FC",
    ":woman_dancing_medium_skin_tone:": "\U0001F483 \U0001F3FD",
    ":woman_dark_skin_tone:": "\U0001F469 \U0001F3FF",
    ":woman_detective:": "\U0001F575 \U0000FE0F \U0000200D \U00002640 \U0000FE0F",
    ":woman_detective_dark_skin_tone:": "\U0001F575 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_detective_light_skin_tone:": "\U0001F575 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_detective_medium-dark_skin_tone:": "\U0001F575 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_detective_medium-light_skin_tone:": "\U0001F575 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_detective_medium_skin_tone:": "\U0001F575 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_facepalming:": "\U0001F926 \U0000200D \U00002640 \U0000FE0F",
    ":woman_facepalming_dark_skin_tone:": "\U0001F926 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_facepalming_light_skin_tone:": "\U0001F926 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_facepalming_medium-dark_skin_tone:": "\U0001F926 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_facepalming_medium-light_skin_tone:": "\U0001F926 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_facepalming_medium_skin_tone:": "\U0001F926 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_factory_worker:": "\U0001F469 \U0000200D \U0001F3ED",
    ":woman_factory_worker_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F3ED",
    ":woman_factory_worker_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F3ED",
    ":woman_factory_worker_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F3ED",
    ":woman_factory_worker_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F3ED",
    ":woman_factory_worker_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F3ED",
    ":woman_farmer:": "\U0001F469 \U0000200D \U0001F33E",
    ":woman_farmer_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F33E",
    ":woman_farmer_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F33E",
    ":woman_farmer_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F33E",
    ":woman_farmer_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F33E",
    ":woman_farmer_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F33E",
    ":woman_firefighter:": "\U0001F469 \U0000200D \U0001F692",
    ":woman_firefighter_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F692",
    ":woman_firefighter_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F692",
    ":woman_firefighter_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F692",
    ":woman_firefighter_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F692",
    ":woman_firefighter_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F692",
    ":woman_frowning:": "\U0001F64D \U0000200D \U00002640 \U0000FE0F",
    ":woman_frowning_dark_skin_tone:": "\U0001F64D \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_frowning_light_skin_tone:": "\U0001F64D \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_frowning_medium-dark_skin_tone:": "\U0001F64D \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_frowning_medium-light_skin_tone:": "\U0001F64D \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_frowning_medium_skin_tone:": "\U0001F64D \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_NO:": "\U0001F645 \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_NO_dark_skin_tone:": "\U0001F645 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_NO_light_skin_tone:": "\U0001F645 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_NO_medium-dark_skin_tone:": "\U0001F645 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_NO_medium-light_skin_tone:": "\U0001F645 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_NO_medium_skin_tone:": "\U0001F645 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_OK:": "\U0001F646 \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_OK_dark_skin_tone:": "\U0001F646 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_OK_light_skin_tone:": "\U0001F646 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_OK_medium-dark_skin_tone:": "\U0001F646 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_OK_medium-light_skin_tone:": "\U0001F646 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_gesturing_OK_medium_skin_tone:": "\U0001F646 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_haircut:": "\U0001F487 \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_haircut_dark_skin_tone:": "\U0001F487 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_haircut_light_skin_tone:": "\U0001F487 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_haircut_medium-dark_skin_tone:": "\U0001F487 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_haircut_medium-light_skin_tone:": "\U0001F487 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_haircut_medium_skin_tone:": "\U0001F487 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_massage:": "\U0001F486 \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_massage_dark_skin_tone:": "\U0001F486 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_massage_light_skin_tone:": "\U0001F486 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_massage_medium-dark_skin_tone:": "\U0001F486 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_massage_medium-light_skin_tone:": "\U0001F486 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_getting_massage_medium_skin_tone:": "\U0001F486 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_golfing:": "\U0001F3CC \U0000FE0F \U0000200D \U00002640 \U0000FE0F",
    ":woman_golfing_dark_skin_tone:": "\U0001F3CC \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_golfing_light_skin_tone:": "\U0001F3CC \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_golfing_medium-dark_skin_tone:": "\U0001F3CC \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_golfing_medium-light_skin_tone:": "\U0001F3CC \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_golfing_medium_skin_tone:": "\U0001F3CC \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_guard:": "\U0001F482 \U0000200D \U00002640 \U0000FE0F",
    ":woman_guard_dark_skin_tone:": "\U0001F482 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_guard_light_skin_tone:": "\U0001F482 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_guard_medium-dark_skin_tone:": "\U0001F482 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_guard_medium-light_skin_tone:": "\U0001F482 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_guard_medium_skin_tone:": "\U0001F482 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_health_worker:": "\U0001F469 \U0000200D \U00002695 \U0000FE0F",
    ":woman_health_worker_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U00002695 \U0000FE0F",
    ":woman_health_worker_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U00002695 \U0000FE0F",
    ":woman_health_worker_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U00002695 \U0000FE0F",
    ":woman_health_worker_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U00002695 \U0000FE0F",
    ":woman_health_worker_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U00002695 \U0000FE0F",
    ":woman_judge:": "\U0001F469 \U0000200D \U00002696 \U0000FE0F",
    ":woman_judge_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U00002696 \U0000FE0F",
    ":woman_judge_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U00002696 \U0000FE0F",
    ":woman_judge_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U00002696 \U0000FE0F",
    ":woman_judge_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U00002696 \U0000FE0F",
    ":woman_judge_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U00002696 \U0000FE0F",
    ":woman_juggling:": "\U0001F939 \U0000200D \U00002640 \U0000FE0F",
    ":woman_juggling_dark_skin_tone:": "\U0001F939 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_juggling_light_skin_tone:": "\U0001F939 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_juggling_medium-dark_skin_tone:": "\U0001F939 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_juggling_medium-light_skin_tone:": "\U0001F939 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_juggling_medium_skin_tone:": "\U0001F939 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_lifting_weights:": "\U0001F3CB \U0000FE0F \U0000200D \U00002640 \U0000FE0F",
    ":woman_lifting_weights_dark_skin_tone:": "\U0001F3CB \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_lifting_weights_light_skin_tone:": "\U0001F3CB \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_lifting_weights_medium-dark_skin_tone:": "\U0001F3CB \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_lifting_weights_medium-light_skin_tone:": "\U0001F3CB \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_lifting_weights_medium_skin_tone:": "\U0001F3CB \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_light_skin_tone:": "\U0001F469 \U0001F3FB",
    ":woman_mechanic:": "\U0001F469 \U0000200D \U0001F527",
    ":woman_mechanic_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F527",
    ":woman_mechanic_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F527",
    ":woman_mechanic_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F527",
    ":woman_mechanic_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F527",
    ":woman_mechanic_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F527",
    ":woman_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE",
    ":woman_medium-light_skin_tone:": "\U0001F469 \U0001F3FC",
    ":woman_medium_skin_tone:": "\U0001F469 \U0001F3FD",
    ":woman_mountain_biking:": "\U0001F6B5 \U0000200D \U00002640 \U0000FE0F",
    ":woman_mountain_biking_dark_skin_tone:": "\U0001F6B5 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_mountain_biking_light_skin_tone:": "\U0001F6B5 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_mountain_biking_medium-dark_skin_tone:": "\U0001F6B5 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_mountain_biking_medium-light_skin_tone:": "\U0001F6B5 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_mountain_biking_medium_skin_tone:": "\U0001F6B5 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_office_worker:": "\U0001F469 \U0000200D \U0001F4BC",
    ":woman_office_worker_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F4BC",
    ":woman_office_worker_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F4BC",
    ":woman_office_worker_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F4BC",
    ":woman_office_worker_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F4BC",
    ":woman_office_worker_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F4BC",
    ":woman_pilot:": "\U0001F469 \U0000200D \U00002708 \U0000FE0F",
    ":woman_pilot_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U00002708 \U0000FE0F",
    ":woman_pilot_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U00002708 \U0000FE0F",
    ":woman_pilot_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U00002708 \U0000FE0F",
    ":woman_pilot_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U00002708 \U0000FE0F",
    ":woman_pilot_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U00002708 \U0000FE0F",
    ":woman_playing_handball:": "\U0001F93E \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_handball_dark_skin_tone:": "\U0001F93E \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_handball_light_skin_tone:": "\U0001F93E \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_handball_medium-dark_skin_tone:": "\U0001F93E \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_handball_medium-light_skin_tone:": "\U0001F93E \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_handball_medium_skin_tone:": "\U0001F93E \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_water_polo:": "\U0001F93D \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_water_polo_dark_skin_tone:": "\U0001F93D \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_water_polo_light_skin_tone:": "\U0001F93D \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_water_polo_medium-dark_skin_tone:": "\U0001F93D \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_water_polo_medium-light_skin_tone:": "\U0001F93D \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_playing_water_polo_medium_skin_tone:": "\U0001F93D \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_police_officer:": "\U0001F46E \U0000200D \U00002640 \U0000FE0F",
    ":woman_police_officer_dark_skin_tone:": "\U0001F46E \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_police_officer_light_skin_tone:": "\U0001F46E \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_police_officer_medium-dark_skin_tone:": "\U0001F46E \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_police_officer_medium-light_skin_tone:": "\U0001F46E \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_police_officer_medium_skin_tone:": "\U0001F46E \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_pouting:": "\U0001F64E \U0000200D \U00002640 \U0000FE0F",
    ":woman_pouting_dark_skin_tone:": "\U0001F64E \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_pouting_light_skin_tone:": "\U0001F64E \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_pouting_medium-dark_skin_tone:": "\U0001F64E \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_pouting_medium-light_skin_tone:": "\U0001F64E \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_pouting_medium_skin_tone:": "\U0001F64E \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_raising_hand:": "\U0001F64B \U0000200D \U00002640 \U0000FE0F",
    ":woman_raising_hand_dark_skin_tone:": "\U0001F64B \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_raising_hand_light_skin_tone:": "\U0001F64B \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_raising_hand_medium-dark_skin_tone:": "\U0001F64B \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_raising_hand_medium-light_skin_tone:": "\U0001F64B \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_raising_hand_medium_skin_tone:": "\U0001F64B \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_rowing_boat:": "\U0001F6A3 \U0000200D \U00002640 \U0000FE0F",
    ":woman_rowing_boat_dark_skin_tone:": "\U0001F6A3 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_rowing_boat_light_skin_tone:": "\U0001F6A3 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_rowing_boat_medium-dark_skin_tone:": "\U0001F6A3 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_rowing_boat_medium-light_skin_tone:": "\U0001F6A3 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_rowing_boat_medium_skin_tone:": "\U0001F6A3 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_running:": "\U0001F3C3 \U0000200D \U00002640 \U0000FE0F",
    ":woman_running_dark_skin_tone:": "\U0001F3C3 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_running_light_skin_tone:": "\U0001F3C3 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_running_medium-dark_skin_tone:": "\U0001F3C3 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_running_medium-light_skin_tone:": "\U0001F3C3 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_running_medium_skin_tone:": "\U0001F3C3 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_scientist:": "\U0001F469 \U0000200D \U0001F52C",
    ":woman_scientist_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F52C",
    ":woman_scientist_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F52C",
    ":woman_scientist_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F52C",
    ":woman_scientist_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F52C",
    ":woman_scientist_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F52C",
    ":woman_shrugging:": "\U0001F937 \U0000200D \U00002640 \U0000FE0F",
    ":woman_shrugging_dark_skin_tone:": "\U0001F937 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_shrugging_light_skin_tone:": "\U0001F937 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_shrugging_medium-dark_skin_tone:": "\U0001F937 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_shrugging_medium-light_skin_tone:": "\U0001F937 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_shrugging_medium_skin_tone:": "\U0001F937 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_singer:": "\U0001F469 \U0000200D \U0001F3A4",
    ":woman_singer_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F3A4",
    ":woman_singer_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F3A4",
    ":woman_singer_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F3A4",
    ":woman_singer_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F3A4",
    ":woman_singer_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F3A4",
    ":woman_student:": "\U0001F469 \U0000200D \U0001F393",
    ":woman_student_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F393",
    ":woman_student_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F393",
    ":woman_student_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F393",
    ":woman_student_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F393",
    ":woman_student_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F393",
    ":woman_surfing:": "\U0001F3C4 \U0000200D \U00002640 \U0000FE0F",
    ":woman_surfing_dark_skin_tone:": "\U0001F3C4 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_surfing_light_skin_tone:": "\U0001F3C4 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_surfing_medium-dark_skin_tone:": "\U0001F3C4 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_surfing_medium-light_skin_tone:": "\U0001F3C4 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_surfing_medium_skin_tone:": "\U0001F3C4 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_swimming:": "\U0001F3CA \U0000200D \U00002640 \U0000FE0F",
    ":woman_swimming_dark_skin_tone:": "\U0001F3CA \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_swimming_light_skin_tone:": "\U0001F3CA \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_swimming_medium-dark_skin_tone:": "\U0001F3CA \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_swimming_medium-light_skin_tone:": "\U0001F3CA \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_swimming_medium_skin_tone:": "\U0001F3CA \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_teacher:": "\U0001F469 \U0000200D \U0001F3EB",
    ":woman_teacher_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F3EB",
    ":woman_teacher_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F3EB",
    ":woman_teacher_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F3EB",
    ":woman_teacher_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F3EB",
    ":woman_teacher_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F3EB",
    ":woman_technologist:": "\U0001F469 \U0000200D \U0001F4BB",
    ":woman_technologist_dark_skin_tone:": "\U0001F469 \U0001F3FF \U0000200D \U0001F4BB",
    ":woman_technologist_light_skin_tone:": "\U0001F469 \U0001F3FB \U0000200D \U0001F4BB",
    ":woman_technologist_medium-dark_skin_tone:": "\U0001F469 \U0001F3FE \U0000200D \U0001F4BB",
    ":woman_technologist_medium-light_skin_tone:": "\U0001F469 \U0001F3FC \U0000200D \U0001F4BB",
    ":woman_technologist_medium_skin_tone:": "\U0001F469 \U0001F3FD \U0000200D \U0001F4BB",
    ":woman_tipping_hand:": "\U0001F481 \U0000200D \U00002640 \U0000FE0F",
    ":woman_tipping_hand_dark_skin_tone:": "\U0001F481 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_tipping_hand_light_skin_tone:": "\U0001F481 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_tipping_hand_medium-dark_skin_tone:": "\U0001F481 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_tipping_hand_medium-light_skin_tone:": "\U0001F481 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_tipping_hand_medium_skin_tone:": "\U0001F481 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_walking:": "\U0001F6B6 \U0000200D \U00002640 \U0000FE0F",
    ":woman_walking_dark_skin_tone:": "\U0001F6B6 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_walking_light_skin_tone:": "\U0001F6B6 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_walking_medium-dark_skin_tone:": "\U0001F6B6 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_walking_medium-light_skin_tone:": "\U0001F6B6 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_walking_medium_skin_tone:": "\U0001F6B6 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman_wearing_turban:": "\U0001F473 \U0000200D \U00002640 \U0000FE0F",
    ":woman_wearing_turban_dark_skin_tone:": "\U0001F473 \U0001F3FF \U0000200D \U00002640 \U0000FE0F",
    ":woman_wearing_turban_light_skin_tone:": "\U0001F473 \U0001F3FB \U0000200D \U00002640 \U0000FE0F",
    ":woman_wearing_turban_medium-dark_skin_tone:": "\U0001F473 \U0001F3FE \U0000200D \U00002640 \U0000FE0F",
    ":woman_wearing_turban_medium-light_skin_tone:": "\U0001F473 \U0001F3FC \U0000200D \U00002640 \U0000FE0F",
    ":woman_wearing_turban_medium_skin_tone:": "\U0001F473 \U0001F3FD \U0000200D \U00002640 \U0000FE0F",
    ":woman’s_boot:": "\U0001F462",
    ":woman’s_clothes:": "\U0001F45A",
    ":woman’s_hat:": "\U0001F452",
    ":woman’s_sandal:": "\U0001F461",
    ":women_with_bunny_ears_partying:": "\U0001F46F \U0000200D \U00002640 \U0000FE0F",
    ":women_wrestling:": "\U0001F93C \U0000200D \U00002640 \U0000FE0F",
    ":women’s_room:": "\U0001F6BA",
    ":world_map:": "\U0001F5FA",
    ":worried_face:": "\U0001F61F",
    ":wrapped_gift:": "\U0001F381",
    ":wrench:": "\U0001F527",
    ":writing_hand:": "\U0000270D",
    ":writing_hand_dark_skin_tone:": "\U0000270D \U0001F3FF",
    ":writing_hand_light_skin_tone:": "\U0000270D \U0001F3FB",
    ":writing_hand_medium-dark_skin_tone:": "\U0000270D \U0001F3FE",
    ":writing_hand_medium-light_skin_tone:": "\U0000270D \U0001F3FC",
    ":writing_hand_medium_skin_tone:": "\U0000270D \U0001F3FD",
    ":yellow_heart:": "\U0001F49B",
    ":yen_banknote:": "\U0001F4B4",
    ":yin_yang:": "\U0000262F",
    ":zipper-mouth_face:": "\U0001F910",
    ":zzz:": "\U0001F4A4",
    ":Åland_Islands:": "\U0001F1E6 \U0001F1FD",
}

UNICODE_EMO = {v: k for k, v in EMO_UNICODE.items()}

In [86]:
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(
            r"(" + emot + ")",
            "_".join(UNICODE_EMO[emot].replace(",", "").replace(":", "").split()),
            text,
        )
    return text


text = "game is on 🔥"
convert_emojis(text)

'game is on fire'

In [87]:
text = "Hilarious 😂"
convert_emojis(text)

'Hilarious face_with_tears_of_joy'

## Removal of URLs

Next preprocessing step is to remove any URLs present in the data. For example, if we are doing a twitter analysis, then there is a good chance that the tweet will have some URL in it. Probably we might need to remove them for our further analysis. 

We can use the below code snippet to do that.

In [88]:
def remove_urls(text):
    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    return url_pattern.sub(r"", text)

Let us take a `https` link and check the code

In [89]:
text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

'Driverless AI NLP blog post on '

Now let us take a `http` url and check the code

In [90]:
text = "Please refer to link http://lnkd.in/ecnt5yC for the paper"
remove_urls(text)

'Please refer to link  for the paper'

Thanks to Pranjal for the edge cases in the comments below. Suppose say there is no `http` or `https` in the url link. The function can now captures that as well.

In [91]:
text = "Want to know more. Checkout www.h2o.ai for additional information"
remove_urls(text)

'Want to know more. Checkout  for additional information'

## Removal of HTML Tags

One another common preprocessing technique that will come handy in multiple places is removal of html tags. This is especially useful, if we scrap the data from different websites. We might end up having html strings as part of our text. 

First, let us try to remove the HTML tags using regular expressions. 

In [92]:
def remove_html(text):
    html_pattern = re.compile("<.*?>")
    return html_pattern.sub(r"", text)


text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

print(remove_html(text))


 H2O
 AutoML
 Driverless AI



We can also use `BeautifulSoup` package to get the text from HTML document in a more elegant way.

In [93]:
def remove_html(text):
    return BeautifulSoup(text, "lxml").text


text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>
"""

print(remove_html(text))


 H2O
 AutoML
 Driverless AI




## Chat Words Conversion

This is an important text preprocessing step if we are dealing with chat data. People do use a lot of abbreviated words in chat and so it might be helpful to expand those words for our analysis purposes. 

Got a good list of chat slang words from this [repo](https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt). We can use this for our conversion here. We can add more words to this list.

In [94]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [ ]:
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)


def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [95]:
chat_words_conversion("one minute BRB")

'one minute Be Right Back'

In [96]:
chat_words_conversion("imo this is awesome")

'In My Opinion this is awesome'

### Spelling Correction

- One another important text preprocessing step is spelling correction. 

- Typos are common in text data and we might want to correct those spelling mistakes before we do our analysis. 

- If we are interested in wrinting a spell corrector of our own, we can probably start with [famous code](https://norvig.com/spell-correct.html) from Peter Norvig.

- In this notebook, let us use the python package `pyspellchecker` for our spelling correction.

In [99]:
spell = SpellChecker()


def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())

    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)


text = "speling correctin"
correct_spellings(text)

'spelling correcting'

In [100]:
text = "thnks for readin the notebook"
correct_spellings(text)

'thanks for reading the notebook'